## 예보데이터 결합
- 지역별/변수별 나누어져 있던 데이터를 연도별로 결합

In [ ]:
def add_mean(path, feature):

  #df = pd.read_csv(path, encoding='cp949')
    df = path

  # 첫행에 ' Start : 20120101 ' 추가(패턴을 맞춰주기위해)
    new = {' format: day' : [' Start : 20160101 '], 'hour' : [np.nan], 'forecast' : [np.nan], 'value  location:65_139 Start : 20160101 ' : [np.nan]}
    #new = {' format: day' : [' Start : 20160101 '], 'hour' : [np.nan], 'forecast' : [np.nan], 'value  location:75_139 Start : 20160101 ' : [np.nan]}
    new = pd.DataFrame(new)
    df1 = pd.concat([new, df])
    df1.reset_index(drop=True, inplace=True)
    
    # 칼럼 별 타입이 다를 수 있음 ---> 문자열로 되어 있는 숫자 칼럼을 수치형으로 변환
    #if df[df.columns[1]].dtypes != 'float64' or df[df.columns[2]].dtypes != 'float64' or df[df.columns[3]].dtypes != 'float64':
    #    df[df.columns[1]] = pd.to_numeric(df[df.columns[1]], errors='coerce')
    #    df[df.columns[2]] = pd.to_numeric(df[df.columns[2]], errors='coerce')
        #df[df.columns[3]] = pd.to_numeric(df[df.columns[3]], errors='coerce')

  # 컬럼명 바꾸기
    #df2 = df1.rename(columns={'format..day':'yymmdd'})
    df2 = df1.rename(columns={' format: day':'yymmdd'})
                              # , 'value  location:65_139 Start : 20120101 ':feature})

  # ' Start : * ' 인덱스 탐색
    idx = df2.index[df2['yymmdd'].str.contains('Start', case=False, na=False)].tolist()

  # 2012____ 리스트에 저장
    dates = []
    for i in range(len(idx)):
        date = df2.iloc[idx[i], 0].split(': ')[1].strip()
        dates.append(date[:-2])
    print(dates)

  # 'yymmdd'(몇일) 리스트에 저장
    days = []
    for i in range(len(idx)):
        if idx[i] == idx[-1]:
            day = list(df2['yymmdd'][idx[i]+1:].map('{}'.format))
    
        else:
            day = list(df2['yymmdd'][idx[i]+1:idx[i+1]].map('{}'.format))
        days.append(day)
    print(days)

  # 'yymmdd':  1,2,3...->20120101, 20120102, 20120103... 바꾸기
    df3 = df2
    num = 0

    for date, day in zip(dates, days):
        j=0
        start = idx[num]+1
        if idx[num] == idx[-1]:
            finish = len(df2)
        else:
            finish = idx[num+1]

        for i in range(start, finish):
            df3.iloc[i, 0] = f'{date}{day[j].strip().zfill(2)}'
            j+=1
        num+=1
  
    idx = df3.index[df3['yymmdd'].str.contains('Start', case=False, na=False)].tolist()
    df3 = df3.drop(idx, axis = 0)
    #df3 = df3.apply(pd.to_numeric)

  # ['yymmdd','hour',]으로 묶고 feature의 평균 구하기
    df4 = df3.groupby(['yymmdd','hour'])[feature].mean()
  
  # ['yymmdd']으로 묶고 feature의 평균 구하기
    df5 = df3.groupby(['yymmdd'])[feature].mean()
  
    return pd.DataFrame(df4), pd.DataFrame(df5)

In [ ]:
area_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
#data_list = ['3시간기온', '6시간강수량'
#             '6시간적설', '12시간강수량', '12시간신적설', '강수형태', '강수확률', 
#             '습도', '일최고기온', '일최저기온', '풍속', '풍향', '하늘상태']

data_list = ['적설', '강수량', '습도', '하늘상태']

for i in range(0, len(area_list)):
    for j in range(0, len(data_list)):
        file_names = glob("2차_과제\\2016\\%s\\*%s*\\*.csv" %(area_list[i], data_list[j])) #폴더 내의 모든 csv파일 목록을 불러온다
        total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

        for file_name in file_names:
            temp = pd.read_csv(file_name) #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
            temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
            total = pd.concat([total, temp], axis=1) #전체 데이터프레임에 추가하여 넣는다
        #48191    
        total_1 = total.iloc[:49590, :2]
        total_2 = total.iloc[:49590, :]
        
        total_2 = total_2.drop([' format: day', 'hour', 'forecast'], axis=1, inplace=False)
        #total_2 = total.drop(['format..day', 'hour', 'forecast'], axis=1, inplace=False)
        
        #total_2 = total.drop(total.iloc[:, :3])
        data = pd.concat([total_1, total_2], axis=1)
        #data = data.apply(pd.to_numeric)
        
        data_1 = data.iloc[:49590, 2:]
        
        mask = data_1.isin(['Start'])
        data_1 = data_1[~mask]
        mask_2 = data_1.isin(['value  location:59_75 Start : 20160101 '])
        data_1 = data_1[~mask_2]
        mask_1 = data_1.isin(['201612format: day'])
        data_1 = data_1[~mask_1]
        data_1 = data_1.apply(pd.to_numeric, errors='ignore')
        data_1 = data_1.astype(float)
        data_mean = data_1.mean(axis=1)
        
        total_data = pd.concat([data.iloc[:, :2], data_mean], axis=1)
        total_data = total_data.rename(columns = {0:'%s'%(data_list[j])})
        
        total_data = total_data[total_data['hour'] != 200.0]
        total_data = total_data[total_data['hour'] != 500.0]
        total_data = total_data[total_data['hour'] != 2000.0]
        total_data = total_data[total_data['hour'] != 2300.0]
        
        feature = '%s'%(data_list[j])
        final1, final2 = add_mean(total_data, feature)
        
        final2['area'] = '%s'%(area_list[i])

        final2.to_csv("2차_데이터_결합\\2016\\%s_%s.csv"%(area_list[i], data_list[j]), encoding='cp949')

In [48]:
import pandas as pd
import numpy as np
from glob import glob

hos = pd.read_csv("back_hospital_.csv")

file_names = glob("2차_데이터_결합\\결합\\20*.csv") #폴더 내의 모든 csv파일 목록을 불러온다
total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
    total = pd.concat([total, temp]) #전체 데이터프레임에 추가하여 넣는다
    
total.to_csv('2차_데이터_결합\\결합\\예보.csv', encoding='cp949')

In [43]:
hos.columns

Index(['Unnamed: 0', 'back_hospital.yyyymmdd', 'back_hospital.area',
       'back_hospital.sex', 'back_hospital.frequency'],
      dtype='object')

In [45]:
a = pd.merge(hos, total,how='inner',on=['yymmdd','area'])
a

,Unnamed: 0_x,yymmdd,area,back_hospital.sex,back_hospital.frequency,Unnamed: 0_y,하늘상태,강수량,습도,적설
0,1055,20120201,강원,1,4,0,1.428571,0.000000,49.782468,0.000000
1,1056,20120201,강원,2,2,0,1.428571,0.000000,49.782468,0.000000
2,1057,20120201,경기,1,3,0,1.727273,0.206349,69.543290,0.000000
3,1058,20120201,경기,2,11,0,1.727273,0.206349,69.543290,0.000000
4,1059,20120201,경남,1,5,0,1.268398,0.722222,50.277056,0.000000
...,...,...,...,...,...,...,...,...,...,...
42053,49670,20151231,제주,2,0,5324,2.810606,0.000000,60.018939,0.000000
42054,49671,20151231,충남,1,0,5679,2.441919,0.000000,72.345960,0.000000
42055,49672,20151231,충남,2,0,5679,2.441919,0.000000,72.345960,0.000000
42056,49673,20151231,충북,1,1,6034,2.393939,0.039216,69.277778,0.039216


In [47]:
a.to_csv('2차_데이터_결합\\결합\\예보_병원.csv', encoding='cp949')

## 예보데이터 결합
- 지역별/변수별 나누어져 있던 데이터를 연도별로 결합

In [48]:
import pandas as pd
import numpy as np
from glob import glob

hos = pd.read_csv("back_hospital_.csv")

file_names = glob("2차_데이터_결합\\결합\\20*.csv") #폴더 내의 모든 csv파일 목록을 불러온다
total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    temp.rename(columns = lambda x: x.split('_')[-1], inplace = True)
    total = pd.concat([total, temp]) #전체 데이터프레임에 추가하여 넣는다
    
total.to_csv('2차_데이터_결합\\결합\\예보.csv', encoding='cp949')

In [43]:
hos.columns

Index(['Unnamed: 0', 'back_hospital.yyyymmdd', 'back_hospital.area',
       'back_hospital.sex', 'back_hospital.frequency'],
      dtype='object')

In [45]:
a = pd.merge(hos, total,how='inner',on=['yymmdd','area'])
a

,Unnamed: 0_x,yymmdd,area,back_hospital.sex,back_hospital.frequency,Unnamed: 0_y,하늘상태,강수량,습도,적설
0,1055,20120201,강원,1,4,0,1.428571,0.000000,49.782468,0.000000
1,1056,20120201,강원,2,2,0,1.428571,0.000000,49.782468,0.000000
2,1057,20120201,경기,1,3,0,1.727273,0.206349,69.543290,0.000000
3,1058,20120201,경기,2,11,0,1.727273,0.206349,69.543290,0.000000
4,1059,20120201,경남,1,5,0,1.268398,0.722222,50.277056,0.000000
...,...,...,...,...,...,...,...,...,...,...
42053,49670,20151231,제주,2,0,5324,2.810606,0.000000,60.018939,0.000000
42054,49671,20151231,충남,1,0,5679,2.441919,0.000000,72.345960,0.000000
42055,49672,20151231,충남,2,0,5679,2.441919,0.000000,72.345960,0.000000
42056,49673,20151231,충북,1,1,6034,2.393939,0.039216,69.277778,0.039216


In [47]:
a.to_csv('2차_데이터_결합\\결합\\예보_병원.csv', encoding='cp949')

## 종관기상관측 데이터 전처리

In [ ]:
import pandas as pd
from glob import glob

In [ ]:
df = pd.read_csv('종관기상관측\\2012\\강원\\강원도_강릉_DAY_2012.csv', encoding='cp949')
df

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
0,105,2012-01-01,0.8,-1.8,505,4.5,1212,NaN,NaN,NaN,...,2.7,2.9,6.3,9.5,14.7,15.9,NaN,NaN,NaN,NaN
1,105,2012-01-02,-0.1,-2.9,807,4.7,1442,NaN,NaN,NaN,...,2.7,2.9,6.2,9.4,14.6,15.8,NaN,NaN,NaN,NaN
2,105,2012-01-03,0.0,-2.9,346,5.5,1416,NaN,NaN,NaN,...,2.7,2.9,6.1,9.2,14.4,15.8,NaN,NaN,NaN,NaN
3,105,2012-01-04,-3.1,-6.3,757,1.8,1405,NaN,NaN,NaN,...,2.6,2.9,6.0,9.2,14.4,15.8,NaN,NaN,NaN,NaN
4,105,2012-01-05,-1.9,-5.7,413,3.5,1449,NaN,NaN,NaN,...,2.5,2.8,6.0,9.0,14.3,15.7,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,105,2012-12-27,-0.1,-5.0,1,6.8,1248,NaN,NaN,NaN,...,2.4,3.0,6.8,10.0,14.8,15.9,NaN,NaN,NaN,NaN
362,105,2012-12-28,0.9,-0.4,5,1.9,2015,NaN,NaN,NaN,...,2.2,2.8,6.6,9.8,14.7,15.9,NaN,NaN,NaN,NaN
363,105,2012-12-29,3.5,1.1,1,5.4,1142,NaN,NaN,NaN,...,2.3,2.7,6.4,9.7,14.6,15.9,NaN,NaN,NaN,NaN
364,105,2012-12-30,-1.1,-7.6,2357,4.1,1,NaN,NaN,NaN,...,2.4,2.8,6.3,9.5,14.6,15.9,NaN,NaN,NaN,NaN


In [ ]:
# 필요한 데이터만 추출
df = df[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '1시간 최다일사량(MJ/m2)', '평균 지면온도(°C)']]
df

,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),1시간 최다일사량(MJ/m2),평균 지면온도(°C)
0,105,2012-01-01,0.8,-1.8,4.5,2.5,-8.6,51.3,3.2,1.70,0.1
1,105,2012-01-02,-0.1,-2.9,4.7,3.1,-7.2,59.4,3.6,1.78,0.1
2,105,2012-01-03,0.0,-2.9,5.5,3.6,-12.2,40.8,2.4,1.75,0.0
3,105,2012-01-04,-3.1,-6.3,1.8,4.5,-15.1,39.3,1.9,1.77,-0.6
4,105,2012-01-05,-1.9,-5.7,3.5,4.0,-17.5,30.0,1.6,1.96,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
361,105,2012-12-27,-0.1,-5.0,6.8,2.5,-18.0,26.1,1.6,1.92,-2.9
362,105,2012-12-28,0.9,-0.4,1.9,1.6,-1.9,83.6,5.5,0.28,-0.4
363,105,2012-12-29,3.5,1.1,5.4,1.4,0.7,83.6,6.5,1.31,0.4
364,105,2012-12-30,-1.1,-7.6,4.1,4.8,-13.8,39.0,2.4,1.88,-0.4


### 1. 이상치 제거
- 0.25 ~ 0.75 범위 사용

In [ ]:
df4 = pd.read_csv('종관기상관측\\2012\\경남\\경남_거제_DAY_2012.csv', encoding='cp949')
df4 = df4[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'

level_1q = df4.quantile(0.25)
print(level_1q,"\n")

level_3q = df4.quantile(0.75)
print(level_3q,"\n")
IQR = level_3q - level_1q

rev_range = 1.5  # 제거 범위 조절 변수
dff = df4[(df4 <= level_3q + (rev_range * IQR)) & (df4 >= level_1q - (rev_range * IQR))]
dff = dff.reset_index(drop=True)
dff[['지점', '일시']] = df4[['지점', '일시']]

dff

지점              294.0
평균기온(°C)          6.6
최저기온(°C)          2.3
최고기온(°C)         10.5
평균 풍속(m/s)        1.4
평균 이슬점온도(°C)     -2.1
평균 상대습도(%)       51.6
평균 증기압(hPa)       5.5
평균 지면온도(°C)       5.9
Name: 0.25, dtype: float64 

지점              294.0
평균기온(°C)         21.5
최저기온(°C)         18.6
최고기온(°C)         26.0
평균 풍속(m/s)        2.4
평균 이슬점온도(°C)     16.1
평균 상대습도(%)       74.9
평균 증기압(hPa)      18.4
평균 지면온도(°C)      24.3
Name: 0.75, dtype: float64 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  if sys.path[0] == '':


,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,294,2012-01-01,2.5,-1.7,7.6,2.1,-5.3,57.3,4.2,1.2
1,294,2012-01-02,1.1,-2.6,6.4,1.5,-9.9,44.1,2.9,0.6
2,294,2012-01-03,1.7,-1.0,5.2,2.2,-7.2,52.4,3.6,0.8
3,294,2012-01-04,-0.2,-2.2,2.7,3.2,-12.0,41.8,2.5,0.4
4,294,2012-01-05,-1.0,-3.2,4.7,1.8,-10.5,50.3,2.8,-0.5
...,...,...,...,...,...,...,...,...,...,...
361,294,2012-12-27,0.6,-5.3,5.6,0.6,-13.4,37.4,2.3,0.3
362,294,2012-12-28,1.2,-0.1,3.8,0.8,-0.3,90.0,6.0,0.1
363,294,2012-12-29,2.3,-1.6,7.6,0.5,0.4,87.8,6.3,0.1
364,294,2012-12-30,0.8,-2.7,3.9,3.4,-7.4,56.9,3.9,0.1


### 2. 결측치 처리

In [ ]:
dff1 = dff.fillna(dff.interpolate()) # 가장 최근 위아래 평균으로 결측치 대입
dff1

,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,295,2012-01-01,1.9,-1.7,6.5,1.90,-5.2,60.0,4.3,0.5
1,295,2012-01-02,-0.9,-6.4,5.0,1.00,-9.2,54.9,3.0,-1.0
2,295,2012-01-03,1.0,-3.8,5.5,1.40,-6.4,58.3,3.8,-0.1
3,295,2012-01-04,-0.6,-1.8,1.9,1.35,-11.0,45.9,2.7,-0.7
4,295,2012-01-05,-0.9,-5.1,4.3,1.30,-9.2,54.3,3.1,-0.9
...,...,...,...,...,...,...,...,...,...,...
361,295,2012-12-27,-0.1,-5.6,4.7,0.90,-13.8,36.4,2.1,-1.7
362,295,2012-12-28,0.6,-1.7,4.0,0.90,-0.8,90.3,5.8,0.1
363,295,2012-12-29,2.4,-0.7,6.2,0.80,0.2,86.0,6.2,0.2
364,295,2012-12-30,0.0,-3.2,3.6,2.30,-7.8,56.9,3.6,0.2


In [ ]:
dff1.isnull().sum()

지점              0
일시              0
평균기온(°C)        0
최저기온(°C)        0
최고기온(°C)        0
평균 풍속(m/s)      0
평균 이슬점온도(°C)    0
평균 상대습도(%)      0
평균 증기압(hPa)     0
평균 지면온도(°C)     0
dtype: int64

### 3. 2012 ~ 2016년, 각 년도의 각 지역으로 통합
- 위의 내용들을 함수로 만들어 한번에 실행되도록 만듬

In [ ]:
def remove_add(area, year):
  # 이상치 제거 & 지역 합치기
  file_names = glob(f'종관기상관측\\{year}\\{area}\\*.csv') #폴더 내의 모든 csv파일 목록을 불러온다
  total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

  for file_name in file_names:
    df4 = pd.read_csv(file_name, encoding='cp949') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    
    # 필요한 변수만 가져옴
    df4 = df4[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'

    # 이상치 처리(0.25~0.75)
    level_1q = df4.quantile(0.25)
    level_3q = df4.quantile(0.75)
    IQR = level_3q - level_1q

    rev_range = 1.5  # 제거 범위 조절 변수
    dff = df4[(df4 <= level_3q + (rev_range * IQR)) & (df4 >= level_1q - (rev_range * IQR))]
    dff = dff.reset_index(drop=True)
    dff[['지점', '일시']] = df4[['지점', '일시']]

    total = pd.concat([total, dff])

  
  # 결측치 처리
  total_N = total.fillna(total.interpolate()) # 가장 최근 위아래 평균으로 결측치 대입


  # ['일시'] 컬럼을 기준으로 그룹화 한 후 데이터 평균 구하기
  total_C = total_N.groupby(['일시']).mean()
  total_C = total_C.reset_index()
  total_C = total_C[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'
    
  total_C['지점'] = area

  return total_C

In [ ]:

area_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '울산', '인천',
            '전남', '전북', '제주', '충남', '충북']
years = ['2012', '2013', '2014', '2015', '2016']

for year in years:
  for area in area_list:
    total_C = remove_add(area, "2015")
    total_C.to_csv(f'종관기상관측_지역별통합\\{year}\\{area}_total.csv', index=False, encoding='cp949') # csv 파일로 저장

## 황사 데이터 전처리

In [ ]:
file_names = glob('과제2\\데이터\\황사\\2012\\강원\\*.csv')
total = pd.DataFrame()

# 강원 지역파일에 있는 모든 .csv파일 concat
for file_name in file_names:
  print(file_name)
  df = pd.read_csv(file_name, encoding='cp949')
  total = pd.concat([total,df])
total

# 각 데이터 프레임의 마지막에 다음해의 첫일이 있음
# 마지막행 => 2013-01-01 00:00

/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_대관령_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_춘천_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_영월_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_속초_HR_2012.csv


,지점,시간,1시간평균 미세먼지농도(㎍/㎥)
0,100,2012-01-01 01:00,48.0
1,100,2012-01-01 02:00,51.0
2,100,2012-01-01 03:00,53.0
3,100,2012-01-01 04:00,46.0
4,100,2012-01-01 05:00,49.0
...,...,...,...
8541,90,2012-12-31 20:00,12.0
8542,90,2012-12-31 21:00,8.0
8543,90,2012-12-31 22:00,12.0
8544,90,2012-12-31 23:00,10.0


In [ ]:
# '시간'컬럼에서 시간(01:00)에 해당하는 부분 제거

splited = total['시간'].str.split(' ', expand=True)
splited.columns = ['일시','시간']
new_df = pd.concat([total, splited], axis=1)
new_df.drop(['시간'], axis=1, inplace=True)
new_df

,지점,1시간평균 미세먼지농도(㎍/㎥),일시
0,100,48.0,2012-01-01
1,100,51.0,2012-01-01
2,100,53.0,2012-01-01
3,100,46.0,2012-01-01
4,100,49.0,2012-01-01
...,...,...,...
8541,90,12.0,2012-12-31
8542,90,8.0,2012-12-31
8543,90,12.0,2012-12-31
8544,90,10.0,2012-12-31


In [ ]:
# 하루 단위로 평균값구하기

new_df1 = new_df.groupby(['일시']).mean()
new_df1 = new_df1.reset_index()
new_df1

,일시,지점,1시간평균 미세먼지농도(㎍/㎥)
0,2012-01-01,103.329114,60.974684
1,2012-01-02,103.000000,67.489583
2,2012-01-03,103.000000,78.437500
3,2012-01-04,103.000000,64.166667
4,2012-01-05,103.000000,46.843750
...,...,...,...
362,2012-12-28,103.000000,36.626374
363,2012-12-29,103.000000,31.086022
364,2012-12-30,103.000000,24.515789
365,2012-12-31,103.000000,18.947917


In [ ]:
# '지점'컬럼의 모든 데이터값을 '강원'으로 바꿈

new_df1['지점'] = '강원'
new_df1 = new_df1[['지점', '일시', '1시간평균 미세먼지농도(㎍/㎥)']]

new_df1 = new_df1.iloc[:-1,:]
new_df1

,지점,일시,1시간평균 미세먼지농도(㎍/㎥)
0,강원,2012-01-01,60.974684
1,강원,2012-01-02,67.489583
2,강원,2012-01-03,78.437500
3,강원,2012-01-04,64.166667
4,강원,2012-01-05,46.843750
...,...,...,...
361,강원,2012-12-27,35.229167
362,강원,2012-12-28,36.626374
363,강원,2012-12-29,31.086022
364,강원,2012-12-30,24.515789


In [ ]:
# 2012 ~ 2016년, 각 년도의 각 지역으로 통합
# 종관기상관측 데이터 전처리와 마찬가지로 위의 내용을 함수로 만들어 한번에 실행시킴

## 학습데이터 생성

### 1. 지역 합치기 & 정렬

In [ ]:
# 지역 합치기 & 정렬

def add_sort(file_names):
  total = pd.DataFrame()

  # 데이터 아래로 합치기
  for file_name in file_names:
    df = pd.read_csv(file_name, encoding='cp949')
    total = pd.concat([total,df])

  # ['일시','지점'] 우선순위로 정렬
  total=total.sort_values(by=[df.columns[1],df.columns[0]])

  # ['일시'] 컬럼 데이터 변경: 2012-01-01 -> 20120101
  # 데이터 타입 변경
  total['일시']= total['일시'].str.replace(pat=r'-',repl=r'',regex=True)
  total['일시'] = pd.to_numeric(total['일시'], errors='coerce')

  # 컬럼명 변경: '일시' -> 'yymmdd' / '지점' -> 'area'
  total = total.rename(columns={'일시':'yyyymmdd', '지점':'area'})

  return total

In [ ]:
# 황사데이터 모든 지역 통합

years = ['2012', '2013', '2014', '2015', '2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\황사\\지역별통합\\{year}\\*')
  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\황사\\total.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,1시간평균 미세먼지농도(㎍/㎥)
0,강원,20120101,60.974684
0,경기,20120101,84.413043
0,경남,20120101,36.260870
0,경북,20120101,48.941176
0,광주,20120101,44.826087
...,...,...,...
364,전남,20161231,15.708333
365,전북,20161231,30.583333
365,제주,20161231,16.434783
365,충남,20161231,40.615385


In [ ]:
# 관측데이터 모든 지역 통합

years = ['2012', '2013', '2014', '2015', '2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\종관기상관측\\지역별통합\\{year}\\*')

  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\종관기상관측\\total.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,강원,20120101,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923
0,경기,20120101,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000
0,경남,20120101,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714
0,경북,20120101,-0.264286,-4.800000,4.621429,3.057143,-7.307143,60.007143,3.607143,-0.192857
0,광주,20120101,1.000000,-2.300000,4.000000,1.800000,-2.500000,77.100000,5.200000,1.700000
...,...,...,...,...,...,...,...,...,...,...
365,전남,20161231,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143
365,전북,20161231,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000
365,제주,20161231,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000
365,충남,20161231,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000


## 백병원과 변수 합치기

In [ ]:
# 백병원 + 황사

hospital = pd.read_csv('과제2\\데이터\\back_hospital_.csv', index_col=0)

# hospital = hospital.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=False)
hospital.columns = ['yyyymmdd', 'area', hospital.columns[2], hospital.columns[3]]

total_C = pd.read_csv('과제2\\데이터\\황사\\total.csv', encoding='cp949')

# 백병원데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 황사 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),back_hospital.sex,back_hospital.frequency
0,20120101,강원,60.974684,1,3
1,20120101,강원,60.974684,2,3
2,20120101,경기,84.413043,1,4
3,20120101,경기,84.413043,2,5
4,20120101,경남,36.260870,1,2
...,...,...,...,...,...
49669,20151231,제주,66.043478,2,0
49670,20151231,충남,44.010870,1,0
49671,20151231,충남,44.010870,2,0
49672,20151231,충북,45.304348,1,1


In [ ]:
# [백병원 + 황사] + 관측
hospital = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\종관기상관측\\total.csv', encoding='cp949')

# [백병원+황사]데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 관측 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature1.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),back_hospital.sex,back_hospital.frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,20151231,제주,66.043478,8.325000,6.075000,10.750000,4.800000,1.425000,62.650000,6.825000,7.200000,2,0
49670,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,1,0
49671,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2,0
49672,20151231,충북,45.304348,0.480000,-3.680000,4.040000,1.480000,-2.960000,79.000000,4.960000,0.820000,1,1


In [ ]:
# [백병원 + 황사 + 관측] + 예보
hospital = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature1.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\예보데이터\\예보.csv', index_col=0, encoding='cp949')
total_C = total_C.drop(['Unnamed: 0.1'], axis=1, inplace=False)
total_C = total_C.rename(columns={'yymmdd':'yyyymmdd'})

# [백병원+황사+관측]데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 예보 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature2.csv', index=False, encoding='cp949')

In [ ]:
# 검증데이터셋에 맞춰 컬럼명 변경
# 'back_hospital.sex' -> 'sex', 'back_hospital.frequency' -> 'frequency'

all_data = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature2.csv', encoding='cp949')
all_data = all_data.rename(columns={'back_hospital.sex':'sex', 'back_hospital.frequency':'frequency'})
all_data.to_csv('과제2\\데이터\\모델링데이터\\test.csv', index=False, encoding='cp949')
all_data

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설,sex,frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,NaN,NaN,NaN,NaN,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50661,20151231,제주,66.043478,8.325000,6.075000,10.750000,4.800000,1.425000,62.650000,6.825000,7.200000,2.810606,0.000000,60.018939,0.000000,2,0
50662,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,1,0
50663,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,2,0
50664,20151231,충북,45.304348,0.480000,-3.680000,4.040000,1.480000,-2.960000,79.000000,4.960000,0.820000,2.393939,0.039216,69.277778,0.039216,1,1


## 2016년도 검증 feature 통합

In [ ]:
# 지역 합치기 & 정렬
# 학습데이터에서 사용한 코드 그대로 사용

def add_sort(file_names):
  total = pd.DataFrame()

  # 데이터 아래로 합치기
  for file_name in file_names:
    df = pd.read_csv(file_name, encoding='cp949')
    total = pd.concat([total,df])

  # ['일시','지점'] 우선순위로 정렬
  total=total.sort_values(by=[df.columns[1],df.columns[0]])

  # ['일시'] 컬럼 데이터 변경: 2012-01-01 -> 20120101
  # 데이터 타입 변경
  total['일시']= total['일시'].str.replace(pat=r'-',repl=r'',regex=True)
  total['일시'] = pd.to_numeric(total['일시'], errors='coerce')

  # 컬럼명 변경: '일시' -> 'yymmdd' / '지점' -> 'area'
  total = total.rename(columns={'일시':'yyyymmdd', '지점':'area'})

  return total

In [ ]:
# 2016년 황사데이터 모든 지역 통합

years = ['2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\황사\\지역별통합\\{year}\\*')
  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\황사\\2016.csv', index=False, encoding='cp949')

In [ ]:
total_C

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥)
0,20160101,강원,45.000000
0,20160101,경기,46.357143
0,20160101,경남,42.952381
0,20160101,경북,34.708861
0,20160101,광주,57.333333
...,...,...,...
364,20161231,전남,15.708333
365,20161231,전북,30.583333
365,20161231,제주,16.434783
365,20161231,충남,40.615385


In [ ]:
# 2016년 관측데이터 모든 지역 통합

years = ['2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\종관기상관측\\지역별통합\\{year}\\*')

  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\종관기상관측\\2016.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,강원,20160101,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769
0,경기,20160101,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000
0,경남,20160101,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714
0,경북,20160101,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571
0,광주,20160101,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000
...,...,...,...,...,...,...,...,...,...,...
365,전남,20161231,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143
365,전북,20161231,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000
365,제주,20161231,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000
365,충남,20161231,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000


In [ ]:
# 2016년 예보 데이터 확인
total_C = pd.read_csv('과제2\\데이터\\예보데이터\\2016.csv', index_col=0, encoding='cp949')
total_C

,yymmdd,강수량,습도,하늘상태,적설,area
0,20160101,0.075630,70.628788,1.079004,0.075630,강원
1,20160102,0.000000,71.925325,1.632035,0.000000,강원
2,20160103,0.000000,56.340909,1.972944,0.000000,강원
3,20160104,0.000000,48.140693,1.991342,0.000000,강원
4,20160105,0.000000,49.071429,1.629870,0.000000,강원
...,...,...,...,...,...,...
361,20161227,0.061404,58.884009,NaN,0.061404,충북
362,20161228,0.175439,60.055180,NaN,0.070175,충북
363,20161229,0.000000,66.476351,NaN,0.000000,충북
364,20161230,0.000000,73.677928,NaN,0.000000,충북


In [ ]:
# yyyymmdd 데이터 형태 바꾸기: 20120101 -> 2012-01-01
def convert(n):
  yy = n//10000
  mm = n%10000//100
  dd = n%100
  l = '{}-{:02d}-{:02d}'.format(yy,mm,dd)
  return l

In [ ]:
# 백병원 + 황사 데이터

hospital = pd.read_csv('과제2\\데이터\\검증데이터\\2-2_검증데이터셋.csv', encoding='cp949')

# yyyymmdd 데이터 형태 바꾸기
# 위에 만든 convert 함수 사용
total_C = pd.read_csv('과제2\\데이터\\황사\\2016.csv', encoding='cp949')
total_C.info()
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

# 두 데이터 결합 후 'sex', 'frequency'컬럼 마지막 열로 보냄
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature.csv', index=False, encoding='cp949')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5482 entries, 0 to 5481
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   yyyymmdd           5482 non-null   int64  
 1   area               5482 non-null   object 
 2   1시간평균 미세먼지농도(㎍/㎥)  5482 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 128.6+ KB


In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),sex,frequency
0,2016-01-01,강원,45.000000,1,NaN
1,2016-01-01,경기,46.357143,1,NaN
2,2016-01-01,경남,42.952381,1,NaN
3,2016-01-01,경북,34.708861,1,NaN
4,2016-01-01,광주,57.333333,1,NaN
...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2,NaN
12440,2016-12-31,전북,30.583333,2,NaN
12441,2016-12-31,제주,16.434783,2,NaN
12442,2016-12-31,충남,40.615385,2,NaN


In [ ]:
# [백병원 + 황사] + 관측

hospital = pd.read_csv('과제2\\데이터\\검증데이터\\hospital_feature.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\종관기상관측\\2016.csv', encoding='cp949')
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature1.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,2,NaN


In [ ]:
# [백병원 + 황사 + 관측] + 예보
hospital = pd.read_csv('과제2\\데이터\\검증데이터\\hospital_feature1.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\예보데이터\\2016.csv', index_col=0, encoding='cp949')
total_C = total_C.rename(columns={'yymmdd':'yyyymmdd'})
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature2.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.075630,70.628788,1.079004,0.07563,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,0.000000,77.372727,3.848485,0.00000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,0.000000,67.110390,3.103896,0.00000,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,0.000000,66.411111,2.793939,0.00000,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,0.000000,72.331081,3.378378,0.00000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,0.000000,68.472973,2.540541,0.00000,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,0.000000,72.510811,2.993243,0.00000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,0.007812,63.423986,2.569820,0.00000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,0.000000,74.515766,NaN,0.00000,2,NaN


## 2016년 대전, 세종 질병 발생 빈도수 평균값 구하기
- 학습데이터에 대전, 세종에 해당하는 학습할 수 있는 데이터가 없음
- 대전, 세종 지역은 질병 발생 빈도수 예측 불가능
- 각 년의 월일에 해당하는 질병 발생 빈도수의 평균값으로 대체

In [ ]:
hospital = pd.read_csv('과제2\\데이터\\back_hospital_.csv', index_col=0)
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49674 entries, 1 to 49674
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   back_hospital.yyyymmdd   49674 non-null  int64 
 1   back_hospital.area       49674 non-null  object
 2   back_hospital.sex        49674 non-null  int64 
 3   back_hospital.frequency  49674 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ MB


In [ ]:
# 문자열로 타입 변경 후 월,일에 해당하는 문자만 추출
hospital['back_hospital.yyyymmdd'] = hospital['back_hospital.yyyymmdd'].astype(str).str[4:]
hospital

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
1,0101,강원,1,3
2,0101,강원,2,3
3,0101,경기,1,4
4,0101,경기,2,5
5,0101,경남,1,2
...,...,...,...,...
49670,1231,제주,2,0
49671,1231,충남,1,0
49672,1231,충남,2,0
49673,1231,충북,1,1


In [ ]:
# 날짜, 지역, 성별을 기준으로 빈도수 평균값 구하기
mean = pd.DataFrame(hospital.groupby(['back_hospital.yyyymmdd', 'back_hospital.area', 'back_hospital.sex'])['back_hospital.frequency'].mean())
mean.reset_index(inplace=True)  # 그룹화 풀기
mean

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
0,0101,강원,1,3.75
1,0101,강원,2,2.75
2,0101,경기,1,4.75
3,0101,경기,2,6.75
4,0101,경남,1,4.50
...,...,...,...,...
12439,1231,제주,2,0.00
12440,1231,충남,1,0.75
12441,1231,충남,2,0.75
12442,1231,충북,1,1.25


In [ ]:
# 대전, 세종 지역만 추출
area = mean[(mean['back_hospital.area']=='대전')|(mean['back_hospital.area']=='세종')]

# 빈도수 반올림
area['back_hospital.frequency'] = area['back_hospital.frequency'].round()
area

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
12,0101,대전,1,2.0
13,0101,대전,2,1.0
18,0101,세종,1,0.0
19,0101,세종,2,0.0
46,0102,대전,1,1.0
...,...,...,...,...
12395,1230,세종,2,0.0
12422,1231,대전,1,1.0
12423,1231,대전,2,0.0
12428,1231,세종,1,0.0


In [ ]:
# 날짜 컬럼의 데이터, 월일 앞에 년(2016) 추가
area['back_hospital.yyyymmdd'] = '2016' + area['back_hospital.yyyymmdd'].astype(str)
area

area.to_csv('과제2\\데이터\\검증데이터\\hospital_mean.csv', index=False, encoding='cp949')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
12,20160101,대전,1,2.0
13,20160101,대전,2,1.0
18,20160101,세종,1,0.0
19,20160101,세종,2,0.0
46,20160102,대전,1,1.0
...,...,...,...,...
12395,20161230,세종,2,0.0
12422,20161231,대전,1,1.0
12423,20161231,대전,2,0.0
12428,20161231,세종,1,0.0


## 종관기상관측 데이터 전처리

In [ ]:
import pandas as pd
from glob import glob

In [ ]:
df = pd.read_csv('과제2\\데이터\\종관기상관측\\2012\\강원\\강원도_강릉_DAY_2012.csv', encoding='cp949')
df

,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
0,105,2012-01-01,0.8,-1.8,505,4.5,1212,NaN,NaN,NaN,...,2.7,2.9,6.3,9.5,14.7,15.9,NaN,NaN,NaN,NaN
1,105,2012-01-02,-0.1,-2.9,807,4.7,1442,NaN,NaN,NaN,...,2.7,2.9,6.2,9.4,14.6,15.8,NaN,NaN,NaN,NaN
2,105,2012-01-03,0.0,-2.9,346,5.5,1416,NaN,NaN,NaN,...,2.7,2.9,6.1,9.2,14.4,15.8,NaN,NaN,NaN,NaN
3,105,2012-01-04,-3.1,-6.3,757,1.8,1405,NaN,NaN,NaN,...,2.6,2.9,6.0,9.2,14.4,15.8,NaN,NaN,NaN,NaN
4,105,2012-01-05,-1.9,-5.7,413,3.5,1449,NaN,NaN,NaN,...,2.5,2.8,6.0,9.0,14.3,15.7,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,105,2012-12-27,-0.1,-5.0,1,6.8,1248,NaN,NaN,NaN,...,2.4,3.0,6.8,10.0,14.8,15.9,NaN,NaN,NaN,NaN
362,105,2012-12-28,0.9,-0.4,5,1.9,2015,NaN,NaN,NaN,...,2.2,2.8,6.6,9.8,14.7,15.9,NaN,NaN,NaN,NaN
363,105,2012-12-29,3.5,1.1,1,5.4,1142,NaN,NaN,NaN,...,2.3,2.7,6.4,9.7,14.6,15.9,NaN,NaN,NaN,NaN
364,105,2012-12-30,-1.1,-7.6,2357,4.1,1,NaN,NaN,NaN,...,2.4,2.8,6.3,9.5,14.6,15.9,NaN,NaN,NaN,NaN


In [ ]:
# 필요한 데이터만 추출
df = df[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '1시간 최다일사량(MJ/m2)', '평균 지면온도(°C)']]
df

,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),1시간 최다일사량(MJ/m2),평균 지면온도(°C)
0,105,2012-01-01,0.8,-1.8,4.5,2.5,-8.6,51.3,3.2,1.70,0.1
1,105,2012-01-02,-0.1,-2.9,4.7,3.1,-7.2,59.4,3.6,1.78,0.1
2,105,2012-01-03,0.0,-2.9,5.5,3.6,-12.2,40.8,2.4,1.75,0.0
3,105,2012-01-04,-3.1,-6.3,1.8,4.5,-15.1,39.3,1.9,1.77,-0.6
4,105,2012-01-05,-1.9,-5.7,3.5,4.0,-17.5,30.0,1.6,1.96,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
361,105,2012-12-27,-0.1,-5.0,6.8,2.5,-18.0,26.1,1.6,1.92,-2.9
362,105,2012-12-28,0.9,-0.4,1.9,1.6,-1.9,83.6,5.5,0.28,-0.4
363,105,2012-12-29,3.5,1.1,5.4,1.4,0.7,83.6,6.5,1.31,0.4
364,105,2012-12-30,-1.1,-7.6,4.1,4.8,-13.8,39.0,2.4,1.88,-0.4


### 1. 이상치 제거
- 0.25 ~ 0.75 범위 사용

In [ ]:
df4 = pd.read_csv('과제2\\데이터\\종관기상관측\\2012\\경남\\경남_거제_DAY_2012.csv', encoding='cp949')
df4 = df4[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'

level_1q = df4.quantile(0.25)
print(level_1q,"\n")

level_3q = df4.quantile(0.75)
print(level_3q,"\n")
IQR = level_3q - level_1q

rev_range = 1.5  # 제거 범위 조절 변수
dff = df4[(df4 <= level_3q + (rev_range * IQR)) & (df4 >= level_1q - (rev_range * IQR))]
dff = dff.reset_index(drop=True)
dff[['지점', '일시']] = df4[['지점', '일시']]

dff

지점              294.0
평균기온(°C)          6.6
최저기온(°C)          2.3
최고기온(°C)         10.5
평균 풍속(m/s)        1.4
평균 이슬점온도(°C)     -2.1
평균 상대습도(%)       51.6
평균 증기압(hPa)       5.5
평균 지면온도(°C)       5.9
Name: 0.25, dtype: float64 

지점              294.0
평균기온(°C)         21.5
최저기온(°C)         18.6
최고기온(°C)         26.0
평균 풍속(m/s)        2.4
평균 이슬점온도(°C)     16.1
평균 상대습도(%)       74.9
평균 증기압(hPa)      18.4
평균 지면온도(°C)      24.3
Name: 0.75, dtype: float64 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  if sys.path[0] == '':


,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,294,2012-01-01,2.5,-1.7,7.6,2.1,-5.3,57.3,4.2,1.2
1,294,2012-01-02,1.1,-2.6,6.4,1.5,-9.9,44.1,2.9,0.6
2,294,2012-01-03,1.7,-1.0,5.2,2.2,-7.2,52.4,3.6,0.8
3,294,2012-01-04,-0.2,-2.2,2.7,3.2,-12.0,41.8,2.5,0.4
4,294,2012-01-05,-1.0,-3.2,4.7,1.8,-10.5,50.3,2.8,-0.5
...,...,...,...,...,...,...,...,...,...,...
361,294,2012-12-27,0.6,-5.3,5.6,0.6,-13.4,37.4,2.3,0.3
362,294,2012-12-28,1.2,-0.1,3.8,0.8,-0.3,90.0,6.0,0.1
363,294,2012-12-29,2.3,-1.6,7.6,0.5,0.4,87.8,6.3,0.1
364,294,2012-12-30,0.8,-2.7,3.9,3.4,-7.4,56.9,3.9,0.1


### 2. 결측치 처리

In [ ]:
dff1 = dff.fillna(dff.interpolate()) # 가장 최근 위아래 평균으로 결측치 대입
dff1

,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,295,2012-01-01,1.9,-1.7,6.5,1.90,-5.2,60.0,4.3,0.5
1,295,2012-01-02,-0.9,-6.4,5.0,1.00,-9.2,54.9,3.0,-1.0
2,295,2012-01-03,1.0,-3.8,5.5,1.40,-6.4,58.3,3.8,-0.1
3,295,2012-01-04,-0.6,-1.8,1.9,1.35,-11.0,45.9,2.7,-0.7
4,295,2012-01-05,-0.9,-5.1,4.3,1.30,-9.2,54.3,3.1,-0.9
...,...,...,...,...,...,...,...,...,...,...
361,295,2012-12-27,-0.1,-5.6,4.7,0.90,-13.8,36.4,2.1,-1.7
362,295,2012-12-28,0.6,-1.7,4.0,0.90,-0.8,90.3,5.8,0.1
363,295,2012-12-29,2.4,-0.7,6.2,0.80,0.2,86.0,6.2,0.2
364,295,2012-12-30,0.0,-3.2,3.6,2.30,-7.8,56.9,3.6,0.2


In [ ]:
dff1.isnull().sum()

지점              0
일시              0
평균기온(°C)        0
최저기온(°C)        0
최고기온(°C)        0
평균 풍속(m/s)      0
평균 이슬점온도(°C)    0
평균 상대습도(%)      0
평균 증기압(hPa)     0
평균 지면온도(°C)     0
dtype: int64

### 3. 2012 ~ 2016년, 각 년도의 각 지역으로 통합
- 위의 내용들을 함수로 만들어 한번에 실행되도록 만듬

In [ ]:
def remove_add(area, year):
  # 이상치 제거 & 지역 합치기
  file_names = glob(f'과제2\\데이터\\종관기상관측\\{year}\\{area}\\*.csv') #폴더 내의 모든 csv파일 목록을 불러온다
  total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

  for file_name in file_names:
    df4 = pd.read_csv(file_name, encoding='cp949') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    
    # 필요한 변수만 가져옴
    df4 = df4[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'

    # 이상치 처리(0.25~0.75)
    level_1q = df4.quantile(0.25)
    level_3q = df4.quantile(0.75)
    IQR = level_3q - level_1q

    rev_range = 1.5  # 제거 범위 조절 변수
    dff = df4[(df4 <= level_3q + (rev_range * IQR)) & (df4 >= level_1q - (rev_range * IQR))]
    dff = dff.reset_index(drop=True)
    dff[['지점', '일시']] = df4[['지점', '일시']]

    total = pd.concat([total, dff])

  
  # 결측치 처리
  total_N = total.fillna(total.interpolate()) # 가장 최근 위아래 평균으로 결측치 대입


  # ['일시'] 컬럼을 기준으로 그룹화 한 후 데이터 평균 구하기
  total_C = total_N.groupby(['일시']).mean()
  total_C = total_C.reset_index()
  total_C = total_C[['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '평균 풍속(m/s)', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 지면온도(°C)']] #, '1시간 최다일사량(MJ/m2)'
    
  total_C['지점'] = area

  return total_C

In [ ]:

area_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '울산', '인천',
            '전남', '전북', '제주', '충남', '충북']
years = ['2012', '2013', '2014', '2015', '2016']

for year in years:
  for area in area_list:
    total_C = remove_add(area, "2015")
    total_C.to_csv(f'과제2\\데이터\\종관기상관측\\지역별통합\\{year}\\{area}_total.csv', index=False, encoding='cp949') # csv 파일로 저장

## 황사 데이터 전처리

In [ ]:
file_names = glob('과제2\\데이터\\황사\\2012\\강원\\*.csv')
total = pd.DataFrame()

# 강원 지역파일에 있는 모든 .csv파일 concat
for file_name in file_names:
  print(file_name)
  df = pd.read_csv(file_name, encoding='cp949')
  total = pd.concat([total,df])
total

# 각 데이터 프레임의 마지막에 다음해의 첫일이 있음
# 마지막행 => 2013-01-01 00:00

/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_대관령_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_춘천_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_영월_HR_2012.csv
/content/drive/MyDrive/Colab Notebooks/기상청공모전/과제2/데이터/황사/2012/강원/강원도_속초_HR_2012.csv


,지점,시간,1시간평균 미세먼지농도(㎍/㎥)
0,100,2012-01-01 01:00,48.0
1,100,2012-01-01 02:00,51.0
2,100,2012-01-01 03:00,53.0
3,100,2012-01-01 04:00,46.0
4,100,2012-01-01 05:00,49.0
...,...,...,...
8541,90,2012-12-31 20:00,12.0
8542,90,2012-12-31 21:00,8.0
8543,90,2012-12-31 22:00,12.0
8544,90,2012-12-31 23:00,10.0


In [ ]:
# '시간'컬럼에서 시간(01:00)에 해당하는 부분 제거

splited = total['시간'].str.split(' ', expand=True)
splited.columns = ['일시','시간']
new_df = pd.concat([total, splited], axis=1)
new_df.drop(['시간'], axis=1, inplace=True)
new_df

,지점,1시간평균 미세먼지농도(㎍/㎥),일시
0,100,48.0,2012-01-01
1,100,51.0,2012-01-01
2,100,53.0,2012-01-01
3,100,46.0,2012-01-01
4,100,49.0,2012-01-01
...,...,...,...
8541,90,12.0,2012-12-31
8542,90,8.0,2012-12-31
8543,90,12.0,2012-12-31
8544,90,10.0,2012-12-31


In [ ]:
# 하루 단위로 평균값구하기

new_df1 = new_df.groupby(['일시']).mean()
new_df1 = new_df1.reset_index()
new_df1

,일시,지점,1시간평균 미세먼지농도(㎍/㎥)
0,2012-01-01,103.329114,60.974684
1,2012-01-02,103.000000,67.489583
2,2012-01-03,103.000000,78.437500
3,2012-01-04,103.000000,64.166667
4,2012-01-05,103.000000,46.843750
...,...,...,...
362,2012-12-28,103.000000,36.626374
363,2012-12-29,103.000000,31.086022
364,2012-12-30,103.000000,24.515789
365,2012-12-31,103.000000,18.947917


In [ ]:
# '지점'컬럼의 모든 데이터값을 '강원'으로 바꿈

new_df1['지점'] = '강원'
new_df1 = new_df1[['지점', '일시', '1시간평균 미세먼지농도(㎍/㎥)']]

new_df1 = new_df1.iloc[:-1,:]
new_df1

,지점,일시,1시간평균 미세먼지농도(㎍/㎥)
0,강원,2012-01-01,60.974684
1,강원,2012-01-02,67.489583
2,강원,2012-01-03,78.437500
3,강원,2012-01-04,64.166667
4,강원,2012-01-05,46.843750
...,...,...,...
361,강원,2012-12-27,35.229167
362,강원,2012-12-28,36.626374
363,강원,2012-12-29,31.086022
364,강원,2012-12-30,24.515789


In [ ]:
# 2012 ~ 2016년, 각 년도의 각 지역으로 통합
# 종관기상관측 데이터 전처리와 마찬가지로 위의 내용을 함수로 만들어 한번에 실행시킴

## 학습데이터 생성

### 1. 지역 합치기 & 정렬

In [ ]:
# 지역 합치기 & 정렬

def add_sort(file_names):
  total = pd.DataFrame()

  # 데이터 아래로 합치기
  for file_name in file_names:
    df = pd.read_csv(file_name, encoding='cp949')
    total = pd.concat([total,df])

  # ['일시','지점'] 우선순위로 정렬
  total=total.sort_values(by=[df.columns[1],df.columns[0]])

  # ['일시'] 컬럼 데이터 변경: 2012-01-01 -> 20120101
  # 데이터 타입 변경
  total['일시']= total['일시'].str.replace(pat=r'-',repl=r'',regex=True)
  total['일시'] = pd.to_numeric(total['일시'], errors='coerce')

  # 컬럼명 변경: '일시' -> 'yymmdd' / '지점' -> 'area'
  total = total.rename(columns={'일시':'yyyymmdd', '지점':'area'})

  return total

In [ ]:
# 황사데이터 모든 지역 통합

years = ['2012', '2013', '2014', '2015', '2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\황사\\지역별통합\\{year}\\*')
  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\황사\\total.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,1시간평균 미세먼지농도(㎍/㎥)
0,강원,20120101,60.974684
0,경기,20120101,84.413043
0,경남,20120101,36.260870
0,경북,20120101,48.941176
0,광주,20120101,44.826087
...,...,...,...
364,전남,20161231,15.708333
365,전북,20161231,30.583333
365,제주,20161231,16.434783
365,충남,20161231,40.615385


In [ ]:
# 관측데이터 모든 지역 통합

years = ['2012', '2013', '2014', '2015', '2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\종관기상관측\\지역별통합\\{year}\\*')

  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\종관기상관측\\total.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,강원,20120101,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923
0,경기,20120101,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000
0,경남,20120101,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714
0,경북,20120101,-0.264286,-4.800000,4.621429,3.057143,-7.307143,60.007143,3.607143,-0.192857
0,광주,20120101,1.000000,-2.300000,4.000000,1.800000,-2.500000,77.100000,5.200000,1.700000
...,...,...,...,...,...,...,...,...,...,...
365,전남,20161231,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143
365,전북,20161231,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000
365,제주,20161231,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000
365,충남,20161231,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000


## 백병원과 변수 합치기

In [ ]:
# 백병원 + 황사

hospital = pd.read_csv('과제2\\데이터\\back_hospital_.csv', index_col=0)

# hospital = hospital.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=False)
hospital.columns = ['yyyymmdd', 'area', hospital.columns[2], hospital.columns[3]]

total_C = pd.read_csv('과제2\\데이터\\황사\\total.csv', encoding='cp949')

# 백병원데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 황사 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),back_hospital.sex,back_hospital.frequency
0,20120101,강원,60.974684,1,3
1,20120101,강원,60.974684,2,3
2,20120101,경기,84.413043,1,4
3,20120101,경기,84.413043,2,5
4,20120101,경남,36.260870,1,2
...,...,...,...,...,...
49669,20151231,제주,66.043478,2,0
49670,20151231,충남,44.010870,1,0
49671,20151231,충남,44.010870,2,0
49672,20151231,충북,45.304348,1,1


In [ ]:
# [백병원 + 황사] + 관측
hospital = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\종관기상관측\\total.csv', encoding='cp949')

# [백병원+황사]데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 관측 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature1.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),back_hospital.sex,back_hospital.frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,20151231,제주,66.043478,8.325000,6.075000,10.750000,4.800000,1.425000,62.650000,6.825000,7.200000,2,0
49670,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,1,0
49671,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2,0
49672,20151231,충북,45.304348,0.480000,-3.680000,4.040000,1.480000,-2.960000,79.000000,4.960000,0.820000,1,1


In [ ]:
# [백병원 + 황사 + 관측] + 예보
hospital = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature1.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\예보데이터\\예보.csv', index_col=0, encoding='cp949')
total_C = total_C.drop(['Unnamed: 0.1'], axis=1, inplace=False)
total_C = total_C.rename(columns={'yymmdd':'yyyymmdd'})

# [백병원+황사+관측]데이터의 'yyyymmdd', 'area' 컬럼을 기준으로 예보 데이터와 결합
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['back_hospital.sex', 'back_hospital.frequency'], axis=1, inplace=False)
total_final['back_hospital.sex'] = total['back_hospital.sex']
total_final['back_hospital.frequency'] = total['back_hospital.frequency']

total_final.to_csv('과제2\\데이터\\모델링데이터\\hospital_feature2.csv', index=False, encoding='cp949')

In [ ]:
# 검증데이터셋에 맞춰 컬럼명 변경
# 'back_hospital.sex' -> 'sex', 'back_hospital.frequency' -> 'frequency'

all_data = pd.read_csv('과제2\\데이터\\모델링데이터\\hospital_feature2.csv', encoding='cp949')
all_data = all_data.rename(columns={'back_hospital.sex':'sex', 'back_hospital.frequency':'frequency'})
all_data.to_csv('과제2\\데이터\\모델링데이터\\test.csv', index=False, encoding='cp949')
all_data

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설,sex,frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,NaN,NaN,NaN,NaN,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50661,20151231,제주,66.043478,8.325000,6.075000,10.750000,4.800000,1.425000,62.650000,6.825000,7.200000,2.810606,0.000000,60.018939,0.000000,2,0
50662,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,1,0
50663,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,2,0
50664,20151231,충북,45.304348,0.480000,-3.680000,4.040000,1.480000,-2.960000,79.000000,4.960000,0.820000,2.393939,0.039216,69.277778,0.039216,1,1


## 2016년도 검증 feature 통합

In [ ]:
# 지역 합치기 & 정렬
# 학습데이터에서 사용한 코드 그대로 사용

def add_sort(file_names):
  total = pd.DataFrame()

  # 데이터 아래로 합치기
  for file_name in file_names:
    df = pd.read_csv(file_name, encoding='cp949')
    total = pd.concat([total,df])

  # ['일시','지점'] 우선순위로 정렬
  total=total.sort_values(by=[df.columns[1],df.columns[0]])

  # ['일시'] 컬럼 데이터 변경: 2012-01-01 -> 20120101
  # 데이터 타입 변경
  total['일시']= total['일시'].str.replace(pat=r'-',repl=r'',regex=True)
  total['일시'] = pd.to_numeric(total['일시'], errors='coerce')

  # 컬럼명 변경: '일시' -> 'yymmdd' / '지점' -> 'area'
  total = total.rename(columns={'일시':'yyyymmdd', '지점':'area'})

  return total

In [ ]:
# 2016년 황사데이터 모든 지역 통합

years = ['2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\황사\\지역별통합\\{year}\\*')
  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\황사\\2016.csv', index=False, encoding='cp949')

In [ ]:
total_C

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥)
0,20160101,강원,45.000000
0,20160101,경기,46.357143
0,20160101,경남,42.952381
0,20160101,경북,34.708861
0,20160101,광주,57.333333
...,...,...,...
364,20161231,전남,15.708333
365,20161231,전북,30.583333
365,20161231,제주,16.434783
365,20161231,충남,40.615385


In [ ]:
# 2016년 관측데이터 모든 지역 통합

years = ['2016']
total_C = pd.DataFrame()

for year in years:
  file_names = glob(f'과제2\\데이터\\종관기상관측\\지역별통합\\{year}\\*')

  total_C = pd.concat([total_C, add_sort(file_names)])

total_C.to_csv('과제2\\데이터\\종관기상관측\\2016.csv', index=False, encoding='cp949')

In [ ]:
total_C

,area,yyyymmdd,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C)
0,강원,20160101,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769
0,경기,20160101,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000
0,경남,20160101,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714
0,경북,20160101,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571
0,광주,20160101,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000
...,...,...,...,...,...,...,...,...,...,...
365,전남,20161231,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143
365,전북,20161231,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000
365,제주,20161231,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000
365,충남,20161231,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000


In [ ]:
# 2016년 예보 데이터 확인
total_C = pd.read_csv('과제2\\데이터\\예보데이터\\2016.csv', index_col=0, encoding='cp949')
total_C

,yymmdd,강수량,습도,하늘상태,적설,area
0,20160101,0.075630,70.628788,1.079004,0.075630,강원
1,20160102,0.000000,71.925325,1.632035,0.000000,강원
2,20160103,0.000000,56.340909,1.972944,0.000000,강원
3,20160104,0.000000,48.140693,1.991342,0.000000,강원
4,20160105,0.000000,49.071429,1.629870,0.000000,강원
...,...,...,...,...,...,...
361,20161227,0.061404,58.884009,NaN,0.061404,충북
362,20161228,0.175439,60.055180,NaN,0.070175,충북
363,20161229,0.000000,66.476351,NaN,0.000000,충북
364,20161230,0.000000,73.677928,NaN,0.000000,충북


In [ ]:
# yyyymmdd 데이터 형태 바꾸기: 20120101 -> 2012-01-01
def convert(n):
  yy = n//10000
  mm = n%10000//100
  dd = n%100
  l = '{}-{:02d}-{:02d}'.format(yy,mm,dd)
  return l

In [ ]:
# 백병원 + 황사 데이터

hospital = pd.read_csv('과제2\\데이터\\검증데이터\\2-2_검증데이터셋.csv', encoding='cp949')

# yyyymmdd 데이터 형태 바꾸기
# 위에 만든 convert 함수 사용
total_C = pd.read_csv('과제2\\데이터\\황사\\2016.csv', encoding='cp949')
total_C.info()
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

# 두 데이터 결합 후 'sex', 'frequency'컬럼 마지막 열로 보냄
total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature.csv', index=False, encoding='cp949')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5482 entries, 0 to 5481
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   yyyymmdd           5482 non-null   int64  
 1   area               5482 non-null   object 
 2   1시간평균 미세먼지농도(㎍/㎥)  5482 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 128.6+ KB


In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),sex,frequency
0,2016-01-01,강원,45.000000,1,NaN
1,2016-01-01,경기,46.357143,1,NaN
2,2016-01-01,경남,42.952381,1,NaN
3,2016-01-01,경북,34.708861,1,NaN
4,2016-01-01,광주,57.333333,1,NaN
...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2,NaN
12440,2016-12-31,전북,30.583333,2,NaN
12441,2016-12-31,제주,16.434783,2,NaN
12442,2016-12-31,충남,40.615385,2,NaN


In [ ]:
# [백병원 + 황사] + 관측

hospital = pd.read_csv('과제2\\데이터\\검증데이터\\hospital_feature.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\종관기상관측\\2016.csv', encoding='cp949')
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature1.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,2,NaN


In [ ]:
# [백병원 + 황사 + 관측] + 예보
hospital = pd.read_csv('과제2\\데이터\\검증데이터\\hospital_feature1.csv', encoding='cp949')

total_C = pd.read_csv('과제2\\데이터\\예보데이터\\2016.csv', index_col=0, encoding='cp949')
total_C = total_C.rename(columns={'yymmdd':'yyyymmdd'})
total_C['yyyymmdd'] = total_C['yyyymmdd'].apply(convert)

total = pd.merge(hospital, total_C, on=["yyyymmdd", "area"], how="left")
total_final = total.drop(['sex', 'frequency'], axis=1, inplace=False)
total_final['sex'] = total['sex']
total_final['frequency'] = total['frequency']

total_final.to_csv('과제2\\데이터\\검증데이터\\hospital_feature2.csv', index=False, encoding='cp949')

In [ ]:
total_final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.075630,70.628788,1.079004,0.07563,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,0.000000,77.372727,3.848485,0.00000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,0.000000,67.110390,3.103896,0.00000,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,0.000000,66.411111,2.793939,0.00000,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,0.000000,72.331081,3.378378,0.00000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,0.000000,68.472973,2.540541,0.00000,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,0.000000,72.510811,2.993243,0.00000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,0.007812,63.423986,2.569820,0.00000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,0.000000,74.515766,NaN,0.00000,2,NaN


## 2016년 대전, 세종 질병 발생 빈도수 평균값 구하기
- 학습데이터에 대전, 세종에 해당하는 학습할 수 있는 데이터가 없음
- 대전, 세종 지역은 질병 발생 빈도수 예측 불가능
- 각 년의 월일에 해당하는 질병 발생 빈도수의 평균값으로 대체

In [ ]:
hospital = pd.read_csv('과제2\\데이터\\back_hospital_.csv', index_col=0)
hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49674 entries, 1 to 49674
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   back_hospital.yyyymmdd   49674 non-null  int64 
 1   back_hospital.area       49674 non-null  object
 2   back_hospital.sex        49674 non-null  int64 
 3   back_hospital.frequency  49674 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ MB


In [ ]:
# 문자열로 타입 변경 후 월,일에 해당하는 문자만 추출
hospital['back_hospital.yyyymmdd'] = hospital['back_hospital.yyyymmdd'].astype(str).str[4:]
hospital

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
1,0101,강원,1,3
2,0101,강원,2,3
3,0101,경기,1,4
4,0101,경기,2,5
5,0101,경남,1,2
...,...,...,...,...
49670,1231,제주,2,0
49671,1231,충남,1,0
49672,1231,충남,2,0
49673,1231,충북,1,1


In [ ]:
# 날짜, 지역, 성별을 기준으로 빈도수 평균값 구하기
mean = pd.DataFrame(hospital.groupby(['back_hospital.yyyymmdd', 'back_hospital.area', 'back_hospital.sex'])['back_hospital.frequency'].mean())
mean.reset_index(inplace=True)  # 그룹화 풀기
mean

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
0,0101,강원,1,3.75
1,0101,강원,2,2.75
2,0101,경기,1,4.75
3,0101,경기,2,6.75
4,0101,경남,1,4.50
...,...,...,...,...
12439,1231,제주,2,0.00
12440,1231,충남,1,0.75
12441,1231,충남,2,0.75
12442,1231,충북,1,1.25


In [ ]:
# 대전, 세종 지역만 추출
area = mean[(mean['back_hospital.area']=='대전')|(mean['back_hospital.area']=='세종')]

# 빈도수 반올림
area['back_hospital.frequency'] = area['back_hospital.frequency'].round()
area

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
12,0101,대전,1,2.0
13,0101,대전,2,1.0
18,0101,세종,1,0.0
19,0101,세종,2,0.0
46,0102,대전,1,1.0
...,...,...,...,...
12395,1230,세종,2,0.0
12422,1231,대전,1,1.0
12423,1231,대전,2,0.0
12428,1231,세종,1,0.0


In [ ]:
# 날짜 컬럼의 데이터, 월일 앞에 년(2016) 추가
area['back_hospital.yyyymmdd'] = '2016' + area['back_hospital.yyyymmdd'].astype(str)
area

area.to_csv('과제2\\데이터\\검증데이터\\hospital_mean.csv', index=False, encoding='cp949')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
12,20160101,대전,1,2.0
13,20160101,대전,2,1.0
18,20160101,세종,1,0.0
19,20160101,세종,2,0.0
46,20160102,대전,1,1.0
...,...,...,...,...
12395,20161230,세종,2,0.0
12422,20161231,대전,1,1.0
12423,20161231,대전,2,0.0
12428,20161231,세종,1,0.0


In [ ]:
import pandas as pd
import numpy as np

In [64]:
data = pd.read_csv('test.csv')
data.head()

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설,sex,frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,NaN,NaN,NaN,NaN,1,2


## 회귀분석

In [46]:
data_drop = data.dropna(axis=0)
data_drop

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설,sex,frequency
1054,20120201,강원,36.739583,-10.576923,-15.669231,-4.530769,2.819231,-20.953846,45.269231,1.192308,-2.600000,1.428571,0.000000,49.782468,0.000000,1,4
1055,20120201,강원,36.739583,-10.576923,-15.669231,-4.530769,2.819231,-20.953846,45.269231,1.192308,-2.600000,1.428571,0.000000,49.782468,0.000000,2,2
1056,20120201,경기,50.708333,-12.260000,-15.980000,-6.840000,2.560000,-20.000000,53.800000,1.280000,-2.820000,1.727273,0.206349,69.543290,0.000000,1,3
1057,20120201,경기,50.708333,-12.260000,-15.980000,-6.840000,2.560000,-20.000000,53.800000,1.280000,-2.820000,1.727273,0.206349,69.543290,0.000000,2,11
1058,20120201,경남,24.875000,-1.650000,-7.007143,4.957143,2.539286,-14.371429,40.000000,2.292857,0.571429,1.268398,0.722222,50.277056,0.000000,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50661,20151231,제주,66.043478,8.325000,6.075000,10.750000,4.800000,1.425000,62.650000,6.825000,7.200000,2.810606,0.000000,60.018939,0.000000,2,0
50662,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,1,0
50663,20151231,충남,44.010870,1.200000,-2.783333,6.000000,1.233333,-1.066667,86.116667,5.683333,2.100000,2.441919,0.000000,72.345960,0.000000,2,0
50664,20151231,충북,45.304348,0.480000,-3.680000,4.040000,1.480000,-2.960000,79.000000,4.960000,0.820000,2.393939,0.039216,69.277778,0.039216,1,1


In [47]:
import statsmodels.api as sm
X = data_drop.drop(['yyyymmdd','area','sex','frequency'], axis=1, inplace=False)
Y = data_drop['frequency']
res = sm.OLS(Y,X).fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:              frequency   R-squared (uncentered):                   0.479
Model:                            OLS   Adj. R-squared (uncentered):              0.478
Method:                 Least Squares   F-statistic:                              2244.
Date:                Sun, 07 Aug 2022   Prob (F-statistic):                        0.00
Time:                        21:24:35   Log-Likelihood:                         -63220.
No. Observations:               31782   AIC:                                  1.265e+05
Df Residuals:                   31769   BIC:                                  1.266e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [49]:
X.corr()

,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설
1시간평균 미세먼지농도(㎍/㎥),1.000000,-0.189596,-0.227172,-0.142575,-0.036304,-0.221241,-0.198475,-0.251649,-0.195698,-0.124703,0.003674,-0.160038,-0.003621
평균기온(°C),-0.189596,1.000000,0.983723,0.982052,-0.085512,0.950379,0.438234,0.915619,0.982015,0.310911,0.022755,0.400671,-0.045187
최저기온(°C),-0.227172,0.983723,1.000000,0.937422,-0.030532,0.963938,0.509864,0.939993,0.960351,0.333175,0.025862,0.429653,-0.040487
최고기온(°C),-0.142575,0.982052,0.937422,1.000000,-0.152928,0.904812,0.351820,0.861985,0.972848,0.271464,0.017360,0.359246,-0.049627
평균 풍속(m/s),-0.036304,-0.085512,-0.030532,-0.152928,1.000000,-0.115775,-0.162226,-0.084297,-0.081356,-0.088369,0.003214,-0.152639,-0.005386
평균 이슬점온도(°C),-0.221241,0.950379,0.963938,0.904812,-0.115775,1.000000,0.691727,0.956184,0.922281,0.373540,0.033793,0.524344,-0.031412
평균 상대습도(%),-0.198475,0.438234,0.509864,0.351820,-0.162226,0.691727,1.000000,0.640497,0.397116,0.360936,0.042120,0.592646,0.011599
평균 증기압(hPa),-0.251649,0.915619,0.939993,0.861985,-0.084297,0.956184,0.640497,1.000000,0.897848,0.379979,0.037051,0.522331,-0.022851
평균 지면온도(°C),-0.195698,0.982015,0.960351,0.972848,-0.081356,0.922281,0.397116,0.897848,1.000000,0.282342,0.017032,0.383207,-0.042881
하늘상태,-0.124703,0.310911,0.333175,0.271464,-0.088369,0.373540,0.360936,0.379979,0.282342,1.000000,0.136209,0.562760,0.037662


#### 지면온도, 강수량 , 적설, 최저,최고빼고(일교차)남기고

### 2016년 결측치확인

In [30]:
yes = pd.read_csv('2016.csv')
yes.isna().sum()

Unnamed: 0      0
yymmdd          0
강수량             0
습도              0
하늘상태          527
적설              0
area            0
dtype: int64

In [38]:
yes2

,Unnamed: 0,yymmdd,강수량,습도,하늘상태,적설,area
0,0,20160101,0.075630,70.628788,1.079004,0.075630,강원
1,1,20160102,0.000000,71.925325,1.632035,0.000000,강원
2,2,20160103,0.000000,56.340909,1.972944,0.000000,강원
3,3,20160104,0.000000,48.140693,1.991342,0.000000,강원
4,4,20160105,0.000000,49.071429,1.629870,0.000000,강원
...,...,...,...,...,...,...,...
6217,361,20161227,0.061404,58.884009,NaN,0.061404,충북
6218,362,20161228,0.175439,60.055180,NaN,0.070175,충북
6219,363,20161229,0.000000,66.476351,NaN,0.000000,충북
6220,364,20161230,0.000000,73.677928,NaN,0.000000,충북


In [41]:
yes2 = yes.sort_values(['area', 'yymmdd'])
yes3 = yes2[yes2['하늘상태'].isna()]
yes3.to_csv('하늘상태.csv', encoding='utf-8-sig')

### 충남 7/24 - 12/31, 충북1 - 12/31  하늘상태 결측치

In [73]:
data_2016 = pd.read_csv('검증데이터.csv')
data_2016

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.075630,70.628788,1.079004,0.07563,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,0.000000,77.372727,3.848485,0.00000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,0.000000,67.110390,3.103896,0.00000,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,0.000000,66.411111,2.793939,0.00000,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,0.000000,72.331081,3.378378,0.00000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,0.000000,68.472973,2.540541,0.00000,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,0.000000,72.510811,2.993243,0.00000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,0.007812,63.423986,2.569820,0.00000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,0.000000,74.515766,NaN,0.00000,2,NaN


### 대전, 세종 제거

In [74]:
data_16 = data_2016[(data_2016['area']!='대전') &(data_2016['area']!='세종')]
data_16

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.075630,70.628788,1.079004,0.07563,1,NaN
1,2016-01-01,경기,46.357143,-0.300000,-5.940000,4.540000,0.800000,-3.680000,79.280000,4.720000,-0.340000,0.000000,77.372727,3.848485,0.00000,1,NaN
2,2016-01-01,경남,42.952381,1.764286,-4.264286,9.057143,1.121429,-4.821429,64.828571,4.314286,1.585714,0.000000,67.110390,3.103896,0.00000,1,NaN
3,2016-01-01,경북,34.708861,1.078571,-5.092857,7.442857,2.100000,-6.035714,62.414286,3.935714,0.128571,0.000000,66.411111,2.793939,0.00000,1,NaN
4,2016-01-01,광주,57.333333,3.200000,-1.500000,8.100000,1.000000,-1.400000,73.100000,5.500000,2.000000,0.000000,72.331081,3.378378,0.00000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,15.708333,2.735714,-2.914286,8.528571,1.471429,-1.407143,75.728571,5.635714,2.057143,0.000000,68.472973,2.540541,0.00000,2,NaN
12440,2016-12-31,전북,30.583333,0.690000,-4.310000,5.930000,0.780000,-2.290000,81.390000,5.280000,0.390000,0.000000,72.510811,2.993243,0.00000,2,NaN
12441,2016-12-31,제주,16.434783,6.700000,3.750000,10.025000,2.600000,0.325000,64.500000,6.275000,6.075000,0.007812,63.423986,2.569820,0.00000,2,NaN
12442,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,0.000000,74.515766,NaN,0.00000,2,NaN


In [75]:
data_16.isna().sum()

yyyymmdd                 0
area                     0
1시간평균 미세먼지농도(㎍/㎥)       16
평균기온(°C)                 0
최저기온(°C)                 0
최고기온(°C)                 0
평균 풍속(m/s)               0
평균 이슬점온도(°C)             0
평균 상대습도(%)               0
평균 증기압(hPa)              0
평균 지면온도(°C)              0
강수량                      0
습도                       0
하늘상태                  1054
적설                       0
sex                      0
frequency            10980
dtype: int64

In [76]:
data_16 = data_16.sort_values(['area','yyyymmdd','sex'])
data_16

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
0,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.07563,70.628788,1.079004,0.07563,1,NaN
17,2016-01-01,강원,45.000000,0.530769,-4.653846,4.930769,1.776923,-6.523077,63.884615,3.869231,-0.330769,0.07563,70.628788,1.079004,0.07563,2,NaN
34,2016-01-02,강원,37.447917,5.176923,1.484615,9.053846,1.957692,-1.107692,66.138462,5.700000,2.192308,0.00000,71.925325,1.632035,0.00000,1,NaN
51,2016-01-02,강원,37.447917,5.176923,1.484615,9.053846,1.957692,-1.107692,66.138462,5.700000,2.192308,0.00000,71.925325,1.632035,0.00000,2,NaN
68,2016-01-03,강원,53.687500,4.915385,0.869231,10.261538,1.407692,0.738462,76.976923,6.523077,2.561538,0.00000,56.340909,1.972944,0.00000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12375,2016-12-29,충북,21.000000,-3.600000,-6.900000,-0.300000,2.640000,-12.040000,55.500000,2.520000,-1.180000,0.00000,66.476351,NaN,0.00000,2,NaN
12392,2016-12-30,충북,21.473684,-3.780000,-9.780000,2.780000,1.500000,-10.300000,62.980000,2.820000,-1.760000,0.00000,73.677928,NaN,0.00000,1,NaN
12409,2016-12-30,충북,21.473684,-3.780000,-9.780000,2.780000,1.500000,-10.300000,62.980000,2.820000,-1.760000,0.00000,73.677928,NaN,0.00000,2,NaN
12426,2016-12-31,충북,21.833333,-0.740000,-5.440000,3.700000,0.840000,-4.020000,78.840000,4.580000,-0.240000,0.00000,72.373874,NaN,0.00000,1,NaN


In [77]:
data_16_1 = data_16[(data_16['area']!='충남') & (data_16['area']!='충북')] # 충남,충북 제외
data_16_2 = data_16[(data_16['area']=='충남')] # 충남 
data_16_3 = data_16[(data_16['area']=='충북')] # 충북

### 충남 7/14~12/31 결측치채우기

In [78]:
data_16_2.isna().sum()

yyyymmdd               0
area                   0
1시간평균 미세먼지농도(㎍/㎥)      0
평균기온(°C)               0
최저기온(°C)               0
최고기온(°C)               0
평균 풍속(m/s)             0
평균 이슬점온도(°C)           0
평균 상대습도(%)             0
평균 증기압(hPa)            0
평균 지면온도(°C)            0
강수량                    0
습도                     0
하늘상태                 322
적설                     0
sex                    0
frequency            732
dtype: int64

In [79]:
data_16_2

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),강수량,습도,하늘상태,적설,sex,frequency
15,2016-01-01,충남,35.542169,1.883333,-4.083333,7.116667,1.216667,-1.250000,80.916667,5.633333,1.833333,0.000000,74.861486,2.932432,0.000000,1,NaN
32,2016-01-01,충남,35.542169,1.883333,-4.083333,7.116667,1.216667,-1.250000,80.916667,5.633333,1.833333,0.000000,74.861486,2.932432,0.000000,2,NaN
49,2016-01-02,충남,33.395833,6.150000,0.916667,12.083333,1.766667,3.316667,83.566667,7.866667,4.350000,0.000000,74.311937,2.815315,0.000000,1,NaN
66,2016-01-02,충남,33.395833,6.150000,0.916667,12.083333,1.766667,3.316667,83.566667,7.866667,4.350000,0.000000,74.311937,2.815315,0.000000,2,NaN
83,2016-01-03,충남,73.739583,6.133333,1.200000,12.216667,0.883333,4.166667,89.100000,8.366667,5.550000,0.000000,67.808559,2.671171,0.000000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,2016-12-29,충남,27.709677,-2.550000,-6.183333,0.900000,1.800000,-8.416667,67.216667,3.333333,0.116667,0.000000,71.441441,NaN,0.000000,2,NaN
12391,2016-12-30,충남,24.563218,-2.666667,-8.916667,3.616667,0.700000,-6.383333,77.450000,3.900000,-0.616667,0.026316,75.238739,NaN,0.026316,1,NaN
12408,2016-12-30,충남,24.563218,-2.666667,-8.916667,3.616667,0.700000,-6.383333,77.450000,3.900000,-0.616667,0.026316,75.238739,NaN,0.026316,2,NaN
12425,2016-12-31,충남,40.615385,1.316667,-2.483333,6.233333,0.683333,-0.516667,88.233333,6.016667,1.600000,0.000000,74.515766,NaN,0.000000,1,NaN


In [84]:
data = pd.read_csv('test.csv')
data.head()

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),평균 지면온도(°C),하늘상태,강수량,습도,적설,sex,frequency
0,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,1,3
1,20120101,강원,60.974684,-2.576923,-6.607692,1.553846,2.115385,-9.200000,62.530769,3.069231,-1.976923,NaN,NaN,NaN,NaN,2,3
2,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,1,4
3,20120101,경기,84.413043,-3.720000,-8.180000,0.720000,1.720000,-8.380000,71.220000,3.320000,-2.260000,NaN,NaN,NaN,NaN,2,5
4,20120101,경남,36.260870,1.157143,-3.250000,6.650000,2.250000,-6.192857,59.228571,3.950000,0.135714,NaN,NaN,NaN,NaN,1,2


In [98]:
data_2 = data[data['area']=='충남']
data_2 = data_2.reset_index()
data_2 = data_2.iloc[:,1:]
data_2['year'] = data_2['yyyymmdd']//10000
data_2['mmdd'] = data_2['yyyymmdd']%10000

In [124]:
nam_total = pd.DataFrame(data_2.groupby(['mmdd','sex'])['하늘상태'].mean())
nam_total = nam_total['하늘상태'].reset_index()
nam_total = nam_total['하늘상태']
nam_total.to_csv('충남_2.csv', encoding='utf-8-sig')

In [125]:
data_16_2 = data_16_2.reset_index()
data_16_2 = data_16_2.iloc[:,1:]
data_16_2.to_csv('충남.csv',encoding='utf-8-sig')

## excel에서 충남 7/14~12/31 결측치 4년평균으로 채우기

In [130]:
data_16_2 = pd.read_csv('충남.csv') # 채우고 파일 불러오기
data_16_2.isna().sum()

Unnamed: 0             0
yyyymmdd               0
area                   0
1시간평균 미세먼지농도(㎍/㎥)      0
평균기온(°C)               0
최저기온(°C)               0
최고기온(°C)               0
평균 풍속(m/s)             0
평균 이슬점온도(°C)           0
평균 상대습도(%)             0
평균 증기압(hPa)            0
평균 지면온도(°C)            0
강수량                    0
습도                     0
하늘상태                   0
적설                     0
sex                    0
frequency            732
dtype: int64

### 충북 1/1~12/31 결측치채우기

In [131]:
data_16_3.isna().sum()

yyyymmdd               0
area                   0
1시간평균 미세먼지농도(㎍/㎥)      4
평균기온(°C)               0
최저기온(°C)               0
최고기온(°C)               0
평균 풍속(m/s)             0
평균 이슬점온도(°C)           0
평균 상대습도(%)             0
평균 증기압(hPa)            0
평균 지면온도(°C)            0
강수량                    0
습도                     0
하늘상태                 732
적설                     0
sex                    0
frequency            732
dtype: int64

In [134]:
data_3 = data[data['area']=='충북']
data_3 = data_3.reset_index()
data_3 = data_3.iloc[:,1:]
data_3['year'] = data_3['yyyymmdd']//10000
data_3['mmdd'] = data_3['yyyymmdd']%10000

In [135]:
buk_total = pd.DataFrame(data_3.groupby(['mmdd','sex'])['하늘상태'].mean())
buk_total = buk_total['하늘상태'].reset_index()
buk_total = buk_total['하늘상태']
buk_total.to_csv('충북_2.csv', encoding='utf-8-sig')

In [136]:
data_16_3 = data_16_3.reset_index()
data_16_3 = data_16_3.iloc[:,1:]
data_16_3.to_csv('충북.csv',encoding='utf-8-sig')

## excel에서 충북 1/1~12/31 결측치 4년평균으로 채우기

In [139]:
data_16_3 = pd.read_csv('충북.csv') # 채우고 파일 불러오기
data_16_3.isna().sum()

Unnamed: 0             0
yyyymmdd               0
area                   0
1시간평균 미세먼지농도(㎍/㎥)      4
평균기온(°C)               0
최저기온(°C)               0
최고기온(°C)               0
평균 풍속(m/s)             0
평균 이슬점온도(°C)           0
평균 상대습도(%)             0
평균 증기압(hPa)            0
평균 지면온도(°C)            0
강수량                    0
습도                     0
하늘상태                   0
적설                     0
sex                    0
frequency            732
dtype: int64

## 충남, 충북 결측치 처리 후 16년 data병합

In [209]:
data_16_total = pd.concat([data_16_1, data_16_2, data_16_3], axis=0)
data_16_total.isna().sum()
data_16_total = data_16_total.iloc[:,0:-1]
data_16_total.isna().sum()

yyyymmdd                 0
area                     0
1시간평균 미세먼지농도(㎍/㎥)       16
평균기온(°C)                 0
최저기온(°C)                 0
최고기온(°C)                 0
평균 풍속(m/s)               0
평균 이슬점온도(°C)             0
평균 상대습도(%)               0
평균 증기압(hPa)              0
평균 지면온도(°C)              0
강수량                      0
습도                       0
하늘상태                     0
적설                       0
sex                      0
frequency            10980
dtype: int64

## data중간에 있는 미세먼지농도 위아래 평균값으로 결측치 대체

In [210]:
data_16_total = data_16_total.interpolate()
data_16_total.isna().sum()

yyyymmdd                 0
area                     0
1시간평균 미세먼지농도(㎍/㎥)        0
평균기온(°C)                 0
최저기온(°C)                 0
최고기온(°C)                 0
평균 풍속(m/s)               0
평균 이슬점온도(°C)             0
평균 상대습도(%)               0
평균 증기압(hPa)              0
평균 지면온도(°C)              0
강수량                      0
습도                       0
하늘상태                     0
적설                       0
sex                      0
frequency            10980
dtype: int64

### 일교차 열 만들기

In [211]:
data_16_total['temp_diff'] = data_16_total['최고기온(°C)'] - data_16_total['최저기온(°C)']

### 지면온도, 강수량 , 적설, 최저,최고온도 제거

In [212]:
data_16_total = data_16_total.drop(['평균 지면온도(°C)','최고기온(°C)','최저기온(°C)', '강수량','적설'], axis=1, inplace=False)
data_16_total

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),습도,하늘상태,sex,frequency,temp_diff
0,2016-01-01,강원,45.000000,0.530769,1.776923,-6.523077,63.884615,3.869231,70.628788,1.079004,1,NaN,9.584615
17,2016-01-01,강원,45.000000,0.530769,1.776923,-6.523077,63.884615,3.869231,70.628788,1.079004,2,NaN,9.584615
34,2016-01-02,강원,37.447917,5.176923,1.957692,-1.107692,66.138462,5.700000,71.925325,1.632035,1,NaN,7.569231
51,2016-01-02,강원,37.447917,5.176923,1.957692,-1.107692,66.138462,5.700000,71.925325,1.632035,2,NaN,7.569231
68,2016-01-03,강원,53.687500,4.915385,1.407692,0.738462,76.976923,6.523077,56.340909,1.972944,1,NaN,9.392308
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2016-12-29,충북,21.000000,-3.600000,2.640000,-12.040000,55.500000,2.520000,66.476351,2.567340,2,NaN,6.600000
728,2016-12-30,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,73.677928,2.386364,1,NaN,12.560000
729,2016-12-30,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,73.677928,2.386364,2,NaN,12.560000
730,2016-12-31,충북,21.833333,-0.740000,0.840000,-4.020000,78.840000,4.580000,72.373874,2.058010,1,NaN,9.140000


In [213]:
def convert(n):
    yy = n.split('-')[0]
    mm = n.split('-')[1]
    dd = n.split('-')[2]
    return int(yy+mm+dd)

In [214]:
data_16_total['yyyymmdd'] = data_16_total['yyyymmdd'].apply(convert)

In [215]:
data_16_total

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),습도,하늘상태,sex,frequency,temp_diff
0,20160101,강원,45.000000,0.530769,1.776923,-6.523077,63.884615,3.869231,70.628788,1.079004,1,NaN,9.584615
17,20160101,강원,45.000000,0.530769,1.776923,-6.523077,63.884615,3.869231,70.628788,1.079004,2,NaN,9.584615
34,20160102,강원,37.447917,5.176923,1.957692,-1.107692,66.138462,5.700000,71.925325,1.632035,1,NaN,7.569231
51,20160102,강원,37.447917,5.176923,1.957692,-1.107692,66.138462,5.700000,71.925325,1.632035,2,NaN,7.569231
68,20160103,강원,53.687500,4.915385,1.407692,0.738462,76.976923,6.523077,56.340909,1.972944,1,NaN,9.392308
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,20161229,충북,21.000000,-3.600000,2.640000,-12.040000,55.500000,2.520000,66.476351,2.567340,2,NaN,6.600000
728,20161230,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,73.677928,2.386364,1,NaN,12.560000
729,20161230,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,73.677928,2.386364,2,NaN,12.560000
730,20161231,충북,21.833333,-0.740000,0.840000,-4.020000,78.840000,4.580000,72.373874,2.058010,1,NaN,9.140000


## 2012-2015 결측치제거

In [164]:
data = pd.read_csv('test.csv')
data_2 = data.dropna(axis=0)

### 일교차 열 만들기

In [165]:
data_2['temp_diff'] = data_2['최고기온(°C)'] - data_2['최저기온(°C)']

C:\Users\MSI\AppData\Local\Temp\ipykernel_13272\4177680881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2['temp_diff'] = data_2['최고기온(°C)'] - data_2['최저기온(°C)']


### 지면온도, 강수량 , 적설, 최저,최고온도 제거

In [166]:
data_2 = data_2.drop(['평균 지면온도(°C)','최고기온(°C)','최저기온(°C)', '강수량','적설'], axis=1, inplace=False)
data_2

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),하늘상태,습도,sex,frequency,temp_diff
1054,20120201,강원,36.739583,-10.576923,2.819231,-20.953846,45.269231,1.192308,1.428571,49.782468,1,4,11.138462
1055,20120201,강원,36.739583,-10.576923,2.819231,-20.953846,45.269231,1.192308,1.428571,49.782468,2,2,11.138462
1056,20120201,경기,50.708333,-12.260000,2.560000,-20.000000,53.800000,1.280000,1.727273,69.543290,1,3,9.140000
1057,20120201,경기,50.708333,-12.260000,2.560000,-20.000000,53.800000,1.280000,1.727273,69.543290,2,11,9.140000
1058,20120201,경남,24.875000,-1.650000,2.539286,-14.371429,40.000000,2.292857,1.268398,50.277056,1,5,11.964286
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50661,20151231,제주,66.043478,8.325000,4.800000,1.425000,62.650000,6.825000,2.810606,60.018939,2,0,4.675000
50662,20151231,충남,44.010870,1.200000,1.233333,-1.066667,86.116667,5.683333,2.441919,72.345960,1,0,8.783333
50663,20151231,충남,44.010870,1.200000,1.233333,-1.066667,86.116667,5.683333,2.441919,72.345960,2,0,8.783333
50664,20151231,충북,45.304348,0.480000,1.480000,-2.960000,79.000000,4.960000,2.393939,69.277778,1,1,7.720000


### 2012 - 2015, 2016 data 통합

In [216]:
final = pd.concat([data_2, data_16_total], axis=0)
final.to_csv('통합원본.csv',encoding='utf-8-sig')

In [217]:
final

,yyyymmdd,area,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),하늘상태,습도,sex,frequency,temp_diff
1054,20120201,강원,36.739583,-10.576923,2.819231,-20.953846,45.269231,1.192308,1.428571,49.782468,1,4.0,11.138462
1055,20120201,강원,36.739583,-10.576923,2.819231,-20.953846,45.269231,1.192308,1.428571,49.782468,2,2.0,11.138462
1056,20120201,경기,50.708333,-12.260000,2.560000,-20.000000,53.800000,1.280000,1.727273,69.543290,1,3.0,9.140000
1057,20120201,경기,50.708333,-12.260000,2.560000,-20.000000,53.800000,1.280000,1.727273,69.543290,2,11.0,9.140000
1058,20120201,경남,24.875000,-1.650000,2.539286,-14.371429,40.000000,2.292857,1.268398,50.277056,1,5.0,11.964286
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,20161229,충북,21.000000,-3.600000,2.640000,-12.040000,55.500000,2.520000,2.567340,66.476351,2,NaN,6.600000
728,20161230,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,2.386364,73.677928,1,NaN,12.560000
729,20161230,충북,21.473684,-3.780000,1.500000,-10.300000,62.980000,2.820000,2.386364,73.677928,2,NaN,12.560000
730,20161231,충북,21.833333,-0.740000,0.840000,-4.020000,78.840000,4.580000,2.058010,72.373874,1,NaN,9.140000


### 정규화

### 정규화 진행하지 않을 colums

In [218]:
final1 = final.iloc[:,[0,1,10,11]]

### 정규화 진행할 columns

In [219]:
final2 = final.iloc[:,[2,3,4,5,6,7,8,9,12]]

In [220]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final2[:] = scaler.fit_transform(final2[:])

C:\Users\MSI\AppData\Local\Temp\ipykernel_13272\2296339732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final2[:] = scaler.fit_transform(final2[:])
C:\Users\MSI\AppData\Local\Temp\ipykernel_13272\2296339732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final2[:] = scaler.fit_transform(final2[:])


In [221]:
final_re = pd.concat([final1,final2],axis=1)
final_re

,yyyymmdd,area,sex,frequency,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),하늘상태,습도,temp_diff
1054,20120201,강원,1,4.0,0.051313,0.082418,0.375517,0.087841,0.350407,0.011637,0.142857,0.381018,0.421531
1055,20120201,강원,2,2.0,0.051313,0.082418,0.375517,0.087841,0.350407,0.011637,0.142857,0.381018,0.421531
1056,20120201,경기,1,3.0,0.072637,0.047059,0.338351,0.106087,0.451843,0.014188,0.242424,0.655770,0.337612
1057,20120201,경기,2,11.0,0.072637,0.047059,0.338351,0.106087,0.451843,0.014188,0.242424,0.655770,0.337612
1058,20120201,경남,1,5.0,0.033202,0.269958,0.335381,0.213750,0.287753,0.043653,0.089466,0.387894,0.456209
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,20161229,충북,2,NaN,0.027287,0.228992,0.349821,0.258345,0.472057,0.050261,0.522447,0.613127,0.230954
728,20161230,충북,1,NaN,0.028010,0.225210,0.186380,0.291628,0.560999,0.058989,0.462121,0.713257,0.481224
729,20161230,충북,2,NaN,0.028010,0.225210,0.186380,0.291628,0.560999,0.058989,0.462121,0.713257,0.481224
730,20161231,충북,1,NaN,0.028559,0.289076,0.091756,0.411751,0.749584,0.110189,0.352670,0.695126,0.337612


In [222]:
final_re['year'] = final_re['yyyymmdd']//10000
final_re['mm'] = final_re['yyyymmdd']%10000//100

In [223]:
final_re

,yyyymmdd,area,sex,frequency,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),하늘상태,습도,temp_diff,year,mm
1054,20120201,강원,1,4.0,0.051313,0.082418,0.375517,0.087841,0.350407,0.011637,0.142857,0.381018,0.421531,2012,2
1055,20120201,강원,2,2.0,0.051313,0.082418,0.375517,0.087841,0.350407,0.011637,0.142857,0.381018,0.421531,2012,2
1056,20120201,경기,1,3.0,0.072637,0.047059,0.338351,0.106087,0.451843,0.014188,0.242424,0.655770,0.337612,2012,2
1057,20120201,경기,2,11.0,0.072637,0.047059,0.338351,0.106087,0.451843,0.014188,0.242424,0.655770,0.337612,2012,2
1058,20120201,경남,1,5.0,0.033202,0.269958,0.335381,0.213750,0.287753,0.043653,0.089466,0.387894,0.456209,2012,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,20161229,충북,2,NaN,0.027287,0.228992,0.349821,0.258345,0.472057,0.050261,0.522447,0.613127,0.230954,2016,12
728,20161230,충북,1,NaN,0.028010,0.225210,0.186380,0.291628,0.560999,0.058989,0.462121,0.713257,0.481224,2016,12
729,20161230,충북,2,NaN,0.028010,0.225210,0.186380,0.291628,0.560999,0.058989,0.462121,0.713257,0.481224,2016,12
730,20161231,충북,1,NaN,0.028559,0.289076,0.091756,0.411751,0.749584,0.110189,0.352670,0.695126,0.337612,2016,12


### dataset 나누기

- 학습데이터 : 2012~2015 ( 201203, 201306, 201409, 201512 제외)  
- 검증데이터 : 201203,201306,201409,201512  
- 테스트데이터 : 2016

In [242]:
test_data = final_re[final_re['year']==2016]
test_data = test_data.drop(['year','mm'], axis=1, inplace=False)
test_data = test_data.sort_values(['yyyymmdd','sex','area'])
test_data = test_data.reset_index()
test_data = test_data.iloc[:,1:]
test_data.to_csv('test_data.csv',encoding='utf-8-sig')
test_data

,yyyymmdd,area,sex,frequency,1시간평균 미세먼지농도(㎍/㎥),평균기온(°C),평균 풍속(m/s),평균 이슬점온도(°C),평균 상대습도(%),평균 증기압(hPa),하늘상태,습도,temp_diff
0,20160101,강원,1,NaN,0.063923,0.315772,0.226082,0.363873,0.571755,0.089512,0.026335,0.670862,0.356283
1,20160101,경기,1,NaN,0.065995,0.298319,0.086022,0.418255,0.754816,0.114262,0.949495,0.764629,0.393881
2,20160101,경남,1,NaN,0.060797,0.341687,0.132104,0.396422,0.582979,0.102459,0.701299,0.621943,0.513197
3,20160101,경북,1,NaN,0.048214,0.327281,0.272401,0.373195,0.554272,0.091446,0.597980,0.612220,0.480204
4,20160101,광주,1,NaN,0.082750,0.371849,0.114695,0.461867,0.681332,0.136954,0.792793,0.694531,0.356929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,20161231,전남,2,NaN,0.019209,0.362095,0.182284,0.461730,0.712587,0.140902,0.513514,0.640888,0.434313
10976,20161231,전북,2,NaN,0.041916,0.319118,0.083154,0.444843,0.779905,0.130553,0.664414,0.697030,0.383803
10977,20161231,제주,2,NaN,0.020318,0.445378,0.344086,0.494862,0.579073,0.159499,0.523273,0.570688,0.217307
10978,20161231,충남,2,NaN,0.057230,0.332283,0.069295,0.478763,0.861276,0.151984,0.444256,0.724906,0.319836


In [239]:
vaild_data = final_re[((final_re['year']==2012) & (final_re['mm']==3)) | ((final_re['year']==2013) & (final_re['mm']==6)) |
                     ((final_re['year']==2014) & (final_re['mm']==9)) | ((final_re['year']==2015) & (final_re['mm']==12))]
vaild_data = vaild_data.drop(['year','mm'], axis=1, inplace=False)
vaild_data.to_csv('vaild_data.csv',encoding='utf-8-sig')

In [240]:
train_data = final_re[((final_re['year']==2012) & (final_re['mm']!=3)) | ((final_re['year']==2013) & (final_re['mm']!=6)) |
                     ((final_re['year']==2014) & (final_re['mm']!=9)) | ((final_re['year']==2015) & (final_re['mm']!=12))]
train_data = train_data.drop(['year','mm'], axis=1, inplace=False)
train_data.to_csv('train_data.csv',encoding='utf-8-sig')

### 모델링

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import TimeSeriesSplit 

from sklearn.metrics import mean_squared_error
from math import sqrt

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### 전처리 x,y로 나누기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('2차과제\\train_data.csv', index_col=0, engine='python')  # 학습데이터
test = pd.read_csv('2차과제\\vaild_data.csv', index_col=0, engine='python')  # 검증데이터 (201203, 201306, 201409, 201512)

In [ ]:
# 더미변수
train = pd.get_dummies(train, columns=['area'])
test = pd.get_dummies(test, columns=['area'])

# 학습 데이터
trainX = train.drop(['frequency','yyyymmdd'], axis=1, inplace=False)
trainY = train['frequency']

# 검증 데이터
testX = test.drop(['frequency','yyyymmdd'], axis=1, inplace=False)
testY = test['frequency']

## 1. GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
cv = 4

paramGrid = {"subsample" : [0.5,0.7,0.9], 'n_estimators' : [5000], 'max_depth':[3], 'min_samples_leaf' : [5], 'min_samples_split':[2,4,6,8,10,12], 'learning_rate' : [0.1]}

model_GBR = GradientBoostingRegressor(random_state=42)


gridsearch_GBR = GridSearchCV(model_GBR, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch_GBR.fit(trainX, trainY)

print(gridsearch_GBR)
print('gridsearch_RDFRCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch_GBR.best_score_))
print('gridsearch_RDFRCV 최적 하이퍼파라미터: ', gridsearch_GBR.best_params_)

scores_df = pd.DataFrame(gridsearch_GBR.cv_results_)
scores_df

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

best_model_GBR = gridsearch_GBR.best_estimator_
best_pred = best_model_GBR.predict(testX)

rmse = sqrt(mean_squared_error(testY, best_pred))

print("훈련 세트 정확도: {:.4f}".format(best_model_GBR.score(trainX, trainY)))
print("테스트 세트 정확도: {:.4f}".format(best_model_GBR.score(testX, testY)))
print('{}\n rmse: {:.4f}'.format(best_model_GBR, rmse))
gb_score = rmse

In [ ]:
real_test = pd.read_csv('2차과제\\test_data.csv', index_col=0, engine='python')  # 2016년도 data 불러오기

area = real_test.iloc[:, 1]
area = pd.DataFrame(area)
real_test = pd.get_dummies(real_test, columns=['area'])

real_testX = real_test.drop(['frequency','yyyymmdd'], axis=1, inplace=False) # frequency 열 제거

real_pred = best_model_GBR.predict(real_testX) # 최종 예측할 X 입력
print(real_pred)

### 2. XGboost

In [ ]:
cv = 4

paramGrid = {"subsample" : [0.5, 0.7, 0.9], 'max_depth' : [3, 5, 7, 9,11],
             'n_estimators' : [20000], 'learning_rate' : [0.01, 0.1, 0.3]}

fit_params={"early_stopping_rounds":300, 
            "eval_metric" : "rmse", 
            "eval_set" : [[testX, testY]]}

model = XGBRegressor(random_state=42, tree_method='exact', gpu_id=0)

gridsearch = GridSearchCV(model, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch.fit(trainX, trainY, **fit_params)

print(model)
print('GridSearchCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch.best_score_))
print('GridSearchCV 최적 하이퍼파라미터: ', gridsearch.best_params_)

scores_df = pd.DataFrame(gridsearch.cv_results_)
scores_df

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

best_model_XGBR = gridsearch.best_estimator_
best_pred = best_model_XGBR.predict(testX)

rmse = sqrt(mean_squared_error(testY, best_pred))

print("훈련 세트 정확도: {:.4f}".format(best_model_XGBR.score(trainX, trainY)))
print("테스트 세트 정확도: {:.4f}".format(best_model_XGBR.score(testX, testY)))
print('{}\n rmse: {:.4f}'.format(best_model_XGBR, rmse))
xgb_score = rmse

In [ ]:
real_test = pd.read_csv('2차과제\\test_data.csv', index_col=0, engine='python')  # 2016년도 data 불러오기

real_testX = real_test.drop(['frequency','yyyymmdd','area'], axis=1, inplace=False) # frequency 열 제거

real_pred = best_model_XGBR.predict(real_testX) # 최종 예측할 X 입력
print(real_pred)

### 3. LGBM

In [ ]:
from lightgbm import LGBMRegressor
cv=4 

paramGrid = {"bagging_fraction" : [0.72], 'num_iterations' : [20000], 'max_depth':[7], 'min_data_in_leaf' : [8], 'learning_rate' : [0.03], 'colsample_bytree':[0.72],
              'scale_pos_weight': [1.5], 'lambda_l1':[0.1], 'lambda_l2':[0.35]}

fit_params={"early_stopping_rounds":300,
            "eval_metric" : "rmse", 
            "eval_set" : [[testX, testY]]}


model_LGBMR = LGBMRegressor(random_state=42)


gridsearch_LGBMR = GridSearchCV(model_LGBMR, paramGrid, verbose=1,
                          cv=TimeSeriesSplit(n_splits=cv).get_n_splits([trainX, trainY]))

gridsearch_LGBMR.fit(trainX, trainY, **fit_params)

print(gridsearch_LGBMR)
print('gridsearch_RDFRCV 최고 평균 정확도 수치: {:.4f}'.format(gridsearch_LGBMR.best_score_))
print('gridsearch_RDFRCV 최적 하이퍼파라미터: ', gridsearch_LGBMR.best_params_)

scores_df = pd.DataFrame(gridsearch_LGBMR.cv_results_)
scores_df

In [ ]:
# 최적의 파라미터로 다시 모델링

best_model_LGBMR = gridsearch_LGBMR.best_estimator_
best_pred = best_model_LGBMR.predict(testX)
rmse = sqrt(mean_squared_error(testY, best_pred)) 

# print('{}\ 예측 정확도: {0:.4f} rmse: {0:.4f}'.format(model, accuracy, rmse))
print('{}\n rmse: {:.4f}'.format(best_model_LGBMR, rmse))
lgb_score = rmse

In [ ]:
real_test = pd.read_csv('2차과제\\test_data.csv', index_col=0, engine='python')  # 2016년도 data 불러오기

area = real_test.iloc[:, 1]
area = pd.DataFrame(area)
real_test = pd.get_dummies(real_test, columns=['area'])

real_testX = real_test.drop(['frequency','yyyymmdd'], axis=1, inplace=False) # frequency 열 제거

real_pred = best_model_LGBMR.predict(real_testX) # 최종 예측할 X 입력
print(real_pred)

### 3가지 앙상블

In [ ]:
pred_gb = best_model_GBR.predict(real_testX)
pred_xgb = best_model_XGBR.predict(real_testX)
pred_lgb = best_model_LGBMR.predict(real_testX)

In [ ]:
total_weight = (1. / gb_score) + (1. / xgb_score) + (1. / lgb_score)
real_pred = (pred_gb * (1. / gb_score) + pred_xgb * (1. / xgb_score) +  pred_lgb * (1. / lgb_score)) / total_weight

In [ ]:
print(real_pred)
#총합이 1이 되게 모델마다 weight를 정함.
# train set 결과인 rmse의 역수에 비례해서 가중치 줌.
# 각 모델의 가중치와 예측값을 곱해서 최종 결과물 산출.

### 3가지 모델 & 앙상블 모델 여기서부터 동일

In [ ]:
fre_df = pd.DataFrame(real_pred, columns=['frequency'])
fre_df.head()

## 예측한 빈도수와 년도, 성별 병합

real_testX = real_test.drop(['frequency'], axis=1, inplace=False) # frequency 열 제거

real_testX = real_testX.reset_index()
real_testX = pd.concat([area, real_testX], axis=1)

real_testX_2 = real_testX[['yyyymmdd','area','sex']]
real_testX_2

final = pd.concat([real_testX_2, fre_df],axis=1)

### 대전, 세종 예측 data 통합

In [ ]:
da_se = pd.read_csv('2차과제\\hospital_mean.csv', engine='python')  

In [ ]:
final_2 = pd.concat([final, da_se], axis=0)
final_2

### 검증양식으로 변경

In [ ]:
final_2 = final_2.sort_values(['yyyymmdd','sex','area']) # 검증양식 순서를 위해 재정렬
fre = final_2['frequency']
fre = fre.reset_index()
fre = fre.iloc[:,1:]
test = pd.read_csv('2차과제\\2-2_검증데이터셋.csv')

final = pd.concat([test,fre],axis=1)
final.to_csv('2차과제\\pred.csv', encoding='utf-8-sig')

### 최고 성능의 모델

- 1) GradientBoosting {'subsample':0.7, 'n_estimators':1000, 'max_depth':3, 'min_samples_leaf':5, 'min_samples_split':2, 
                    'learning_rate':0.1}
- 2) XGboost {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators':20000,'subsample'
            0.7}  
- 3) LGMB (bagging_fraction=0.72, colsample_bytree=0.72, lambda_l1=0.1,
              lambda_l2=0.35, learning_rate=0.01, max_depth=7,
              min_data_in_leaf=8, num_iterations=20000, random_state=42,
              scale_pos_weight=1.5)

### 1,2,3번 최적 모델을 이용해 앙상블

In [ ]:
pred_gb = best_model_GBR.predict(real_testX)
pred_xgb = best_model_XGBR.predict(real_testX)
pred_lgb = best_model_LGBMR.predict(real_testX)

total_weight = (1. / gb_score) + (1. / xgb_score) + (1. / lgb_score)
real_pred = (pred_gb * (1. / gb_score) + pred_xgb * (1. / xgb_score) +  pred_lgb * (1. / lgb_score)) / total_weight

print(real_pred)

### real_pred를 위에 방법처럼 검증양식으로 변경